# **Libraries**

In [ ]:
    #GENERIC
import warnings
import pandas as pd
from datetime import datetime, timedelta
#import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from random import random
import math
import time
import os
import pickle
from joblib import load
from tqdm import tqdm
tqdm.pandas()
import xlsxwriter
import numpy as np
    #EIKON SPECIFIC
import eikon as ek
import refinitiv.dataplatform as rdp 
    #DATA EXPLORATION SPECIFIC
import holidays
from pandas.tseries.holiday import USFederalHolidayCalendar
from nltk.corpus import stopwords
from collections import Counter
    #PRE PROCESSING
import re
import nltk  
import collections
from nltk.corpus import stopwords
    #Embedding + Classification
from gensim.models import KeyedVectors
import operator
from transformers import BertTokenizer, BertForSequenceClassification
    #Investment
from itertools import product
from scipy.stats import kurtosis, skew, norm, chi2
import statsmodels.api as sm

In [ ]:
#Change Directory to the OS Folder
os.chdir(r"")

# **EIKON & WRDS Data Retrieval**

Links: https://community.developers.refinitiv.com/questions/32320/how-to-set-sources-for-get-news-headlines.html || https://community.developers.refinitiv.com/questions/28201/how-to-get-list-of-leaver-companies-of-the-asx-100.html || 

## **Credential configuration**

In [ ]:
#Connect to eikon with acess key
with open('0)Eikon_Credentials/Eikon_Api_Key_1.txt', 'r') as f:
    api_key = f.read()
ek.set_app_key(api_key)

In [ ]:
#Connect to eikon with acess key (Library)
with open('0)Eikon_Credentials/Eikon_Library_Api_Key.txt', 'r') as f:
    api_library_key = f.read()
rdp.open_desktop_session(api_library_key) 

## **S&P Historical List**

In [ ]:
#Current S&P List
constituents_fields = ['TR.IndexConstituentRIC','TR.IndexConstituentName','TR.IndexConstituentComName']
Current_SP, e = ek.get_data(['.SPX'], fields=constituents_fields, parameters={'SDate':'2022-04-01'})

In [ ]:
#Changes to S&P
SP_changes_fields = ['TR.IndexJLConstituentChangeDate','TR.IndexJLConstituentRIC.change','TR.IndexJLConstituentName', 'TR.IndexJLConstituentRIC','TR.IndexJLConstituentComName']
SP_changes_df, e = ek.get_data(['.SPX'], fields=SP_changes_fields, parameters={'SDate':'0D', 'EDate':'-2CY', 'IC':'B'})
SP_changes_df["Date"] =  pd.to_datetime(SP_changes_df["Date"]).apply(lambda x: x.date())
SP_changes_df = SP_changes_df[(pd.to_datetime(SP_changes_df["Date"])<=datetime(2022,4,1)) & (pd.to_datetime(SP_changes_df["Date"])>=datetime(2020,9,1))].copy()
#List of Leavers and Joiners
leavers_RIC = list(SP_changes_df[SP_changes_df["Change"]=="Leaver"]["Constituent RIC"])
joiners_RIC = list(SP_changes_df[SP_changes_df["Change"]!="Leaver"]["Constituent RIC"])

In [ ]:
#Check for repeated changes to same company
company_change_frequency = SP_changes_df.groupby(["Constituent RIC", "Constituent Name"]).size()
display(company_change_frequency[company_change_frequency > 1])
display( SP_changes_df[SP_changes_df["Constituent RIC"]=="ZIMV.OQ"])

> **Notes**

Company changes where checked for the cases in which a company changed multiple times and found to be companies that entered for a short time in the S&P and then left. So no instances of a company exiting and entering multiple times.

In [ ]:
#Historical S&P (by joining list of changes and current constituents)
SP_changes_df_noduplicates = SP_changes_df[["Constituent RIC","Constituent Name"]].drop_duplicates()
Historical_SP = pd.concat([Current_SP[["Constituent RIC","Constituent Name"]], SP_changes_df_noduplicates], axis=0)

In [ ]:
#Check Duplicate RIC
list_duplicates = list(Historical_SP[Historical_SP.duplicated("Constituent RIC")]["Constituent RIC"].unique())
for dup in list_duplicates:
    #print(Historical_SP[Historical_SP["Constituent RIC"]==dup])
    pass
    
#Drop duplicates
Historical_SP.drop_duplicates(subset=["Constituent RIC"], inplace=True)

In [ ]:
#Check for repeated changes to same company (to highlight inconsistencies)
company_frequency = Historical_SP.groupby(["Constituent RIC", "Constituent Name"]).size()
display(company_frequency[company_frequency > 1])

In [ ]:
#Check NANS
display(Historical_SP[Historical_SP['Constituent RIC'].isna()])
#Correct NANS by hand
Historical_SP.loc[266, "Constituent RIC"] = "EMBC.O"

In [ ]:
# (timeline as defined by user)
recent_date_chosen = datetime(2022,4,1)
oldest_date_chosen = datetime(2020,9,1)
#REgister times of joining and leaving
Historical_SP["leave"] = Historical_SP.apply(lambda x : SP_changes_df[ (SP_changes_df["Constituent RIC"]==x["Constituent RIC"]) & (SP_changes_df["Change"]=="Leaver") ]["Date"].values[0] if x["Constituent RIC"] in leavers_RIC else recent_date_chosen, axis=1)
Historical_SP["join"] = Historical_SP.apply(lambda x : SP_changes_df[ (SP_changes_df["Constituent RIC"]==x["Constituent RIC"])  & (SP_changes_df["Change"]=="Joiner") ]["Date"].values[0] if x["Constituent RIC"] in joiners_RIC else oldest_date_chosen, axis=1)
#Save Excel version
#Historical_SP.to_excel("1)Eikon_Data/Historical_SP_Constituents.xlsx") 

In [ ]:
display(len(Historical_SP))
Historical_SP.head()

## **Request Headlines**

In [ ]:
Historical_SP_Full = pd.read_excel("1)Eikon_Data/Historical_SP_Constituents.xlsx", index_col=0) 
display(Historical_SP_Full.shape)
display(Historical_SP_Full.head(2))
    #Changes (according to needs)
    #Historical_SP = Historical_SP_Full[(Historical_SP_Full["Constituent RIC"]!="MCO.N") & (Historical_SP_Full["Constituent RIC"]!="IVZ.N")].copy()
display(Historical_SP.shape)

In [ ]:
    #Current date
datestring_time = datetime.strftime(datetime.now(),"%m_%d_%Y__%H_%M")
start_time = time.time()#TIMER of code
safety_start_time = time.time() #Safety TIMER 

company_headlines = []
for index, row in tqdm(Historical_SP.iterrows(), total=Historical_SP.shape[0]):
    #(auxiliary)
    retriving_data =  True # reser individual company loop 
    company_request_loop = 0 # tracer reset
    #Setting up timeline of data to retrieve (based on leave and join datess and if not existing fixed dates)
    join_date = row["join"]
    leave_date = row["leave"]
    print("Start Date",join_date,"  End Date",leave_date)
    while retriving_data :   #Loop to bypass request limit
        try:             
            #Timer to avoid overload of requests
            time.sleep(random() * 3)
            # Make request of news (Note: 100 is maximum per request)
            df = ek.get_news_headlines('R:'+str(row["Constituent RIC"])+' AND Language:LEN', date_from = str(join_date), date_to=str(leave_date), count=100)
            #Add RIC to headline retrieved
            df["Constituent RIC"]= row["Constituent RIC"]
            df["Constituent Name"]= row["Constituent Name"]
            # Add healines to main list
            company_headlines.append(df)
            company_request_loop +=1 #tracker  add       
            # Condition to break loop (if either no news available anymore or if the oldest date matches the start date of the timeline stipulated)
            if (df.shape[0]==0) or (leave_date == df.tail(1).index.item()):
                retriving_data=False
            else:
                # change date of the request of news to end when the existing data already retrieved beggins
                leave_date = df.tail(1).index.item() - timedelta(milliseconds=1)
                
        except Exception as exc:
            print(f"Encountered an unknown error: {exc}")
            pd.concat(company_headlines).to_csv("1)Eikon_Data/" + datestring_time + "_Eikon_headlines.csv")
            print(f"Current end date is: {leave_date}")
            
            #Safety timer every 2 hours asks if code is to be kept running or not (to account for timeout errors from eikon)
            lapsed_time = time.time() - safety_start_time
            if lapsed_time>=3600:
                safety_start_time = time.time()
                safety_question = input("Keep the code running?")
            
    # Traker of iterations
    print("Done with company:", str(row["Constituent RIC"]),"total loops:",str(company_request_loop))
    
#Savind data
pd.concat(company_headlines).to_csv("1)Eikon_Data/" + datestring_time + "_Eikon_headlines.csv")
print("Code ran for --- %s seconds ---" % (time.time() - start_time))

In [ ]:
#Check Data Retrieved (& Combine hadlines from diferent retrieval processes)
EIKON_Headlines = pd.DataFrame()
data_files_list = os.listdir("1)Eikon_Data/Headlines_partitioned_versions/")
for file_name in data_files_list:
    temp_df = pd.read_csv("1)Eikon_Data/Headlines_partitioned_versions/" + file_name) 
    print(f" \t File {file_name} has a shape of {temp_df.shape}")
    display(temp_df.head(2))
    EIKON_Headlines = pd.concat([EIKON_Headlines, temp_df])
display(EIKON_Headlines.head(5))
#(cleaning)
EIKON_Headlines.drop_duplicates(inplace=True)
#EIKON_Headlines = EIKON_Headlines[(EIKON_Headlines["Unnamed: 0"]>="2020-12-31") & (EIKON_Headlines["Unnamed: 0"]<="2022-04-01")].copy()

## **Request Lookup Table with Ticker, RIc,..**

In [ ]:
#Get symbols equivalence
Historical_SP_Full_0 = pd.read_excel("1)Eikon_Data/Historical_SP_Constituents.xlsx", index_col=0) 
RIC_list = list(Historical_SP_Full_0["Constituent RIC"].unique())
symbols_df = rdp.convert_symbols(RIC_list, from_symbol_type=rdp.SymbolTypes.RIC)
display(symbols_df.head(5))
#Add Relevant Data to Existing Historical S&P table (CUSIP)
lookup_symbol_table = Historical_SP_Full_0.merge(symbols_df[["RIC","TickerSymbol","CUSIP"]], how="left", left_on="Constituent RIC", right_on="RIC")
lookup_symbol_table.drop_duplicates(inplace=True)
#lookup_symbol_table[["RIC","TickerSymbol","CUSIP","Constituent Name"]].to_excel("1)Eikon_Data/Symbols_lookup.xlsx", index=0)

* WRDS Library of Identifiers

In [ ]:
WRDS_raw_data = pd.read_csv("2)WRDS_Data/WRDS_symbols_vlookup.csv")
    # Type, Col name and other corretion
WRDS_raw_data["date"] = pd.to_datetime(WRDS_raw_data["date"].astype(str))   
WRDS_raw_data.rename(columns={'PERMNO': 'PERMNO_W', 'date': 'Date', 'TICKER': 'Ticker_W', 'COMNAM': 'Company_W','CUSIP':'CUSIP_W'}, inplace=True)
WRDS_raw_data.drop_duplicates(subset=["Ticker_W","Company_W","CUSIP_W"], inplace=True)

In [ ]:
#Check changes in ticker & cusip combo (merge points)
ticker_cusip_frequency = WRDS_raw_data.groupby(["Ticker_W","CUSIP_W"]).size()
display(ticker_cusip_frequency[ticker_cusip_frequency > 1])
display( WRDS_raw_data[WRDS_raw_data["Ticker_W"]=="AFK"])

In [ ]:
#WRDS_raw_data.to_excel("2)WRDS_Data/WRDS_symbols_vlookup.xlsx", index=0)
    # Note: Some company name changes occourred, after merging data check relevant S&P cases

* Check Against WRDS ticker & CUSIP (VERSION 1)

In [ ]:
    #Check if tickers match the ones in WRDS
        #wrds data
WRDS_symbols_df = pd.read_excel("2)WRDS_Data/WRDS_symbols_vlookup.xlsx") 
WRDS_symbols_df["CUSIP_W"] = WRDS_symbols_df["CUSIP_W"].astype(str)
        #eikon (SP) data
symbols_df = pd.read_excel("1)Eikon_Data/Symbols_lookup__ManualRevision.xlsx") 
symbols_df = symbols_df.rename(columns={'CUSIP': 'CUSIP_E', 'Constituent Name': 'Company_E','RIC':'RIC_E', 'TickerSymbol': 'Ticker_E'})

    #Filtering Data (based on ticker & cusip)  - Only to make code faster
#SP_ticker_list = list(symbols_df["Ticker_E"].unique())
#SP_cusip_list = [str(cusip)[:8] for cusip in symbols_df["CUSIP_E"].unique()]
#WRDS_symbols_SP = WRDS_symbols_df[(WRDS_symbols_df["Ticker_W"].isin(SP_ticker_list)) | (WRDS_symbols_df.apply(lambda x: x["CUSIP_W"] in SP_cusip_list,axis=1))].copy()

    #Aggregating the RIC based on ticker and CUSIP
        #adapting cusip (to same length as WRDS)
symbols_df_w_cussip_header = symbols_df.copy()
symbols_df_w_cussip_header["CUSIP_E"] = symbols_df_w_cussip_header.apply(lambda x: str(x["CUSIP_E"])[:8], axis=1) 
        #aggregating
merged_symbols = symbols_df_w_cussip_header.merge(WRDS_symbols_df, how="left", left_on=["Ticker_E","CUSIP_E"], right_on=["Ticker_W","CUSIP_W"])
merged_symbols = merged_symbols[['RIC_E', 'Ticker_E', 'Ticker_W', 'CUSIP_E', 'CUSIP_W', 'Company_E','Company_W']]
    #Column signaling match of values (ticker & cusip)
merged_symbols["CUSIPTicker_Match"] = np.where(((merged_symbols["Ticker_E"]==merged_symbols["Ticker_W"]) & (merged_symbols["CUSIP_E"]==merged_symbols["CUSIP_W"])),1,0)
    #Checking no match
merged_symbols[merged_symbols["CUSIPTicker_Match"]==0]

In [ ]:
#Save Ticker list
SP_ticker_list = list(symbols_df["Ticker_E"].unique())
with open('2)WRDS_Data/tickers_for_returns_wrds.txt', 'w') as f:
    for item in SP_ticker_list:
        f.write("%s\n" % item)

## GICS Sectors

In [ ]:
#Get RIC symbols
symbols_df = pd.read_excel("1)Eikon_Data/Symbols_lookup__ManualRevision.xlsx") 
print("Shape of Eikon Symbol Lookup Table", symbols_df.shape)
#For each get corresponding sector
GICS_df = pd.DataFrame()
for ric in list(symbols_df["RIC"].unique()):
    df, err = ek.get_data(ric,
                      ['TR.RICCode',
                      'TR.CompanyName',
                      'TR.GICSSector', 
                      'TR.GICSIndustryGroup'])
    GICS_df = pd.concat([GICS_df,df])
display(GICS_df)

In [ ]:
#GICS_df.to_excel("1)Eikon_Data/Sectors_lookup.xlsx", index=0)

## RETURNS (Wrds)

Return Data > https://wrds-www.wharton.upenn.edu/

Using the List of Tickers Compiled in 2.4.1 make a request for the relevant timeline (with some leeway) + quick check

In [ ]:
#Get Return Data (& eikon revised symbol data to check against)
WRDS_RET_Raw_data = pd.read_csv("2)WRDS_Data/WRDS_returns_RAW.csv") 
EIKON_symbols_check = pd.read_excel("1)Eikon_Data/Symbols_lookup__ManualRevision.xlsx") 
print("WRDS Raw Data Size", WRDS_RET_Raw_data.shape)

#Safety Measures and Corrections for Comparability betwenn Eikon Wrds
    #(adjust CUSIP of Eikon to same Wrds Size)
EIKON_symbols_check["CUSIP"] = EIKON_symbols_check.apply(lambda x: str(x["CUSIP"])[:8], axis=1) 

#Merged Table
WrdsRet_EikonSymbol = WRDS_RET_Raw_data.merge(EIKON_symbols_check[["RIC","TickerSymbol","CUSIP","Constituent Name"]], how="left", left_on=["TICKER","CUSIP"], right_on=["TickerSymbol","CUSIP"])
WrdsRet_EikonSymbol = WrdsRet_EikonSymbol[~WrdsRet_EikonSymbol.isnull()[["RIC","TickerSymbol","CUSIP","TICKER"]].any(axis=1)].copy()
print("WRDS Clean Data Size", WrdsRet_EikonSymbol.shape)
print("TIckers in Returns Not in S&P", list(set(WRDS_RET_Raw_data["TICKER"]) - set(EIKON_symbols_check["TickerSymbol"])))
print("")

# Calculating Market Value  (outstanding shares*price)
WrdsRet_EikonSymbol["MktCap"] = WrdsRet_EikonSymbol["SHROUT"]*WrdsRet_EikonSymbol["PRC"]
    #(check nans)
nan_chcek_mkt = WrdsRet_EikonSymbol[WrdsRet_EikonSymbol.isnull()[["MktCap"]].any(axis=1)] 
print("Mkt Cap nan check", len(nan_chcek_mkt))
display(nan_chcek_mkt)
#Note: Fill nans with last valid observations and if not available thenext valid one 
    #(correct nan)
WrdsRet_EikonSymbol.sort_values(by=['RIC','date'], inplace=True)
WrdsRet_EikonSymbol["MktCap"] = WrdsRet_EikonSymbol.groupby("RIC")["MktCap"].transform(lambda x: x.fillna(method="ffill"))
WrdsRet_EikonSymbol["MktCap"] = WrdsRet_EikonSymbol.groupby("RIC")["MktCap"].transform(lambda x: x.fillna(method="bfill"))
print("Mkt Cap nan check", len(WrdsRet_EikonSymbol[WrdsRet_EikonSymbol.isnull()[["MktCap"]].any(axis=1)] ))

#Save Clean Data
WrdsRet_EikonSymbol.to_csv("2)WRDS_Data/WRDS_returns_Clean.csv")

In [ ]:
#Check not keppt values
not_keept = pd.concat([WRDS_RET_Raw_data,WrdsRet_EikonSymbol[list(WRDS_RET_Raw_data.columns)]]).drop_duplicates(keep=False)
print("Statistics on return data drooped")
print("\t Companies")
display(pd.DataFrame(not_keept.groupby(["TICKER"]).size()))
print("\t Date")
display(pd.DataFrame(not_keept.groupby(["date"]).size()))

In [ ]:
#Check companies and date availability
Wrds_Companies_Available = WrdsRet_EikonSymbol.groupby(["RIC","TICKER","CUSIP","COMNAM","Constituent Name"]).agg(
                                                        **{'Max date':pd.NamedAgg(column='date', aggfunc='max'),
                                                        'Min date':pd.NamedAgg(column='date', aggfunc='min'),})
unique_wrds_companies = Wrds_Companies_Available.reset_index()["TICKER"].unique()
unique_eikon_companies = EIKON_symbols_check["TickerSymbol"].unique()
print(f"Companies Available {len(unique_wrds_companies)}  &  Required {len(unique_eikon_companies)} ")
print()
# Check missing companies
print(" \t Missing Ticker")
print(set(EIKON_symbols_check["TickerSymbol"]) - set(Wrds_Companies_Available.reset_index()["TICKER"]))
display(Wrds_Companies_Available)

In [ ]:
# Check Extreme dates available against the Ones needed
    #Relevant DaAta
Historical_SP_Full = pd.read_excel("1)Eikon_Data/Historical_SP_Constituents.xlsx", index_col=0) #leave/join dates
Wrds_Companies_Available_Checks = Wrds_Companies_Available.reset_index().merge(Historical_SP_Full[['Constituent RIC', 'leave', 'join']], how="left", left_on=["RIC"], right_on=["Constituent RIC"])
    #Datetime Conversions for coparision
Wrds_Companies_Available_Checks[['Max date', 'Min date']] = Wrds_Companies_Available_Checks[['Max date', 'Min date']].apply(lambda x : pd.to_datetime(x.astype(str)))
#Check column 1 means GOOD 0 means BAD
Wrds_Companies_Available_Checks["Leave Check"] = np.where(Wrds_Companies_Available_Checks["Max date"]+timedelta(days=1)>=Wrds_Companies_Available_Checks["leave"],1,0)
Wrds_Companies_Available_Checks["Join Check"] = np.where(Wrds_Companies_Available_Checks["Min date"]<=Wrds_Companies_Available_Checks["join"],1,0)
#Check Cases with mismatces
Wrds_Companies_Available_Checks[(Wrds_Companies_Available_Checks["Leave Check"]!=1) | (Wrds_Companies_Available_Checks["Join Check"]!=1)]

In [ ]:
#Check specific Case
ticker_to_check = 'AGN'
print(" \t Infro From EIKON")
display(EIKON_symbols_check[EIKON_symbols_check["TickerSymbol"].str.contains(ticker_to_check)])
print(" \t Results From WRDS")
display(WRDS_RET_Raw_data[WRDS_RET_Raw_data["TICKER"].str.contains(ticker_to_check)].drop_duplicates(subset=["TICKER","CUSIP"]))

# Data Exploration

## Headline Data

* Add Dates to Headline Data

In [ ]:
 #Dataset from Eikon
eikon_news_data_0 = pd.read_csv("1)Eikon_Data/Eikon_headlines_1Set2020_31Mar2022.csv")
display(eikon_news_data_0.shape)
eikon_news_data_0.head(3)

In [ ]:
# CHECK for inconsistencies
    #Merge Data of Headlines with entry S&P entry/exit dates
Historical_SP_Full = pd.read_excel("1)Eikon_Data/Historical_SP_Constituents.xlsx", index_col=0) #leave/join dates
eikon_news_data_1_check = eikon_news_data_0.merge(Historical_SP_Full, how="left", on=["Constituent RIC"], suffixes=('_Headlines', '_SP'))
print("Companies Available in NEWS" , eikon_news_data_1_check["Constituent RIC"].nunique(), " &  Required", Historical_SP_Full["Constituent RIC"].nunique() )
print(" \t Extra or Non Matching:")
extra_RIC = list(set(eikon_news_data_1_check["Constituent RIC"].unique()) - set(Historical_SP_Full["Constituent RIC"].unique()))
missing_RIC = list(set(Historical_SP_Full["Constituent RIC"].unique()) -  set(eikon_news_data_1_check["Constituent RIC"].unique()))
print("RICS In News not in S&P", sorted(extra_RIC))
print("RICS In S&P not in News", sorted(missing_RIC))

#Check NANs
cols_to_check_null = [col for col in eikon_news_data_1_check.columns if col not in ['Constituent Name_Headlines', 'Constituent Name_SP']]
nans_check_headline = eikon_news_data_1_check[eikon_news_data_1_check.isnull()[cols_to_check_null].any(axis=1)]
print(" \t Companies with no correspondence")
RIC_list_for_adjustments = nans_check_headline.drop_duplicates(subset=["Constituent RIC"])
display(RIC_list_for_adjustments)

#Save
#RIC_list_for_adjustments[["Constituent RIC","Constituent Name_Headlines"]].to_excel("1)Eikon_Data/Headlines_RIC_for_Correction.xlsx", index=0)

#Help in checking specific cases (for the Nans)
eikon_unique_ComRic = eikon_news_data_1_check.groupby(["Constituent RIC","Constituent Name_Headlines"]).agg(
                                                **{ 'Max date':pd.NamedAgg(column='versionCreated', aggfunc='max'),
                                                    'Min date':pd.NamedAgg(column='versionCreated', aggfunc='min')}).reset_index()

display(eikon_unique_ComRic[eikon_unique_ComRic["Constituent Name_Headlines"].str.contains("VIA")])
display(eikon_unique_ComRic[eikon_unique_ComRic["Constituent RIC"].str.contains("PAR")])

#check headlines of Nan  RIC
study_df = eikon_news_data_1_check[eikon_news_data_1_check["Constituent RIC"]=='HP.N']
for index, row in study_df.iterrows():
    print(row["text"])

In [ ]:
# ACTUAL MERGE
    #Data REquired
eikon_news_data_01 = eikon_news_data_0.copy()
Historical_SP_Full = pd.read_excel("1)Eikon_Data/Historical_SP_Constituents.xlsx", index_col=0) #leave/join dates
Headlines_Corrected_RIC = pd.read_excel("1)Eikon_Data/Headlines_RIC_for_Correction__ManualRevision.xlsx") #ric corrected
ric_for_correction = list(Headlines_Corrected_RIC[~Headlines_Corrected_RIC.isnull()[["Corrected_RIC","Note"]].any(axis=1)]["Constituent RIC"].unique())
    #Correction to Headline Data
eikon_news_data_01["Constituent RIC"] = eikon_news_data_01.apply(lambda x: Headlines_Corrected_RIC[Headlines_Corrected_RIC["Constituent RIC"]==x["Constituent RIC"]]["Corrected_RIC"].values[0] if x["Constituent RIC"] in ric_for_correction else x["Constituent RIC"], axis=1)
    #Merge
eikon_news_data_02 = eikon_news_data_01.merge(Historical_SP_Full, how="inner", on=["Constituent RIC"], suffixes=('_Headlines', '_SP'))
    
# CHECK RIC Time Boundaries in S&P
eikon_news_data_03 = eikon_news_data_02[(pd.to_datetime(eikon_news_data_02["versionCreated"]).dt.tz_localize(None)>=pd.to_datetime(eikon_news_data_02["join"])) & (pd.to_datetime(eikon_news_data_02["versionCreated"]).dt.tz_localize(None)<=pd.to_datetime(eikon_news_data_02["leave"]))].copy()

# INFO
eikon_news_data_03
display(eikon_news_data_03.shape)
print("Unique RIC", eikon_news_data_03["Constituent RIC"].nunique())
print("RICS Not Available", list(set(Historical_SP_Full["Constituent RIC"].unique())-set(eikon_news_data_03["Constituent RIC"].unique())))
eikon_news_data_03.head(3)

* Create Auxiliary columns

In [ ]:
#Adjust data for better interpretation
eikon_news_data_1 = eikon_news_data_03.copy()
  #Create Date and Datetime columns based on versionCreated
eikon_news_data_1["DateTime"] = pd.to_datetime(eikon_news_data_1["versionCreated"])
eikon_news_data_1["Date"] =  pd.to_datetime(eikon_news_data_1['versionCreated']).dt.normalize()
eikon_news_data_1["leave"] = pd.to_datetime(eikon_news_data_1["leave"])
eikon_news_data_1["join"] = pd.to_datetime(eikon_news_data_1["join"])
  #Add days in S&P
eikon_news_data_1["days"] = (eikon_news_data_1["leave"] - eikon_news_data_1["join"]).dt.days
  #Renaming columns
eikon_news_data_1.rename(columns = {"Constituent RIC": "RIC", "Constituent Name_SP": "Company"}, inplace = True) 
  #Cleaning the sources
eikon_news_data_1["sourceCode"] = eikon_news_data_1["sourceCode"].apply(lambda x: x[3:])
#Creating additional columns to identify holidays and weekends
us_holidays = holidays.US()
eikon_news_data_1["Holiday"] = eikon_news_data_1.apply(lambda x : 1 if x["DateTime"] in us_holidays else 0, axis=1)
eikon_news_data_1["Weekday"]= eikon_news_data_1["Date"].dt.day_name()

* Statistics on the Data

In [ ]:
 #Checking timeline 
print("Data is available from", eikon_news_data_1["Date"].min(), "to", eikon_news_data_1["Date"].max())
fig, ax1 = plt.subplots()
eikon_news_data_1.groupby([(eikon_news_data_1["DateTime"])])['DateTime'].count().plot(ax=ax1, kind='line')
plt.plot()

In [ ]:
 #Checking Sources
fig, ax2 = plt.subplots()
eikon_news_data_1.groupby([(eikon_news_data_1["sourceCode"])])['sourceCode'].count().nlargest(10).plot(ax=ax2, kind='bar')
plt.plot()

In [ ]:
 #Checking Top companies
fig, ax3 = plt.subplots()
eikon_news_data_1.groupby([(eikon_news_data_1["Company"])])['Company'].count().nlargest(10).plot(ax=ax3, kind='bar')
plt.plot()

In [ ]:
 #Checking Weekdays & holidays
print("Percentage of news in holidays is {:.2%} ".format(len(eikon_news_data_1[eikon_news_data_1["Holiday"]==1])/len(eikon_news_data_1)))
fig, ax3 = plt.subplots()
eikon_news_data_1.groupby([(eikon_news_data_1["Weekday"])])['Weekday'].count().nlargest(10).plot(ax=ax3, kind='bar')
plt.plot()

* Checking suspicious data Manually

In [ ]:
#eikon_news_data_1.columns

In [ ]:
#study_df = eikon_news_data_1[eikon_news_data_1["sourceCode"]=='TRANS']    #EDG    #GLFILE   #TRANS
#for index, row in study_df.iterrows():
#    print(row["text"])

In [ ]:
#study_df = eikon_news_data_1[eikon_news_data_1["RIC"]=='MCO.N']
#for index, row in study_df.iterrows():
#    print(row["text"])


> **Notes**

*   Sources

The headlines provided by **EDG** (Edgar Fillings News) & **GLFILE** (Global Fillings) & **TRANS** (Event Transcript News) are just informative of the day in which a company´s fillings, earnings, other events occor having relevant infromation (***Remove***).

*   Companies

The headlines provided by **MCO** (Moody´s) is primarly about other companies (***Remove***).

## Final Data Cleaning

* Remove Irrelevant Sources ...

In [ ]:
eikon_news_data_2 = eikon_news_data_1.copy()
print("Initial News", eikon_news_data_2.shape)
#Eliminating irrelevant news sources
eikon_news_data_2 = eikon_news_data_2[(eikon_news_data_2["sourceCode"]!="EDG") & (eikon_news_data_2["sourceCode"]!="GLFILE") & (eikon_news_data_2["sourceCode"]!="TRANS")].copy()
print("News after Source Elimination", eikon_news_data_2.shape)
#Eliminating irrelevant companies
eikon_news_data_2 = eikon_news_data_2[(eikon_news_data_2["RIC"]!="MCO.N")].copy()
print("News after Company Elimination", eikon_news_data_2.shape)

#Elimianting duplicate (cases with same headline, for same company), keeping the first time instance in which they show up
eikon_news_data_3 = eikon_news_data_2.sort_values(by=['DateTime'])  #from older to newer
    #Keep only the first  repeated new on any given company
eikon_news_data_final = eikon_news_data_3.drop_duplicates(subset=['text', 'RIC'], keep='first') #,'sourceCode'
print("News after Duplicate Elimination (by Ric and text)", eikon_news_data_final.shape)
    
#Signal Headlines common to multiple companies (e.g: industry level deadlines)
eikon_news_data_final['Duplicate_Title'] = eikon_news_data_final.groupby('text')['text'].transform('count')
print("Unique NEws", len(eikon_news_data_final[eikon_news_data_final['Duplicate_Title']==1]))

In [ ]:
save_data = input("Save the Data?  ")
if save_data.lower() == "yes":
    eikon_news_data_final.to_pickle("1)Eikon_Data/Clean_Eikon_headlines_1Set2020_31Mar2022.pkl")  

## Statistics

In [ ]:
#Load Saved Clean Dataframe
eikon_news_data_final = pd.read_pickle("1)Eikon_Data/Clean_Eikon_headlines_1Set2020_31Mar2022.pkl") 

In [ ]:
print("News Data Size", eikon_news_data_final.shape)
#Checking timeline 
print("Data is available from", eikon_news_data_final["Date"].min(), "to", eikon_news_data_final["Date"].max())
fig, ax1 = plt.subplots()
eikon_news_data_final.groupby([(eikon_news_data_final["DateTime"])])['DateTime'].count().plot(ax=ax1, kind='line')
plt.plot()
 #Checking Sources
fig, ax2 = plt.subplots()
eikon_news_data_final.groupby([(eikon_news_data_final["sourceCode"])])['sourceCode'].count().nlargest(10).plot(ax=ax2, kind='bar')
plt.plot()
 #Checking Top companies
fig, ax3 = plt.subplots()
eikon_news_data_final.groupby([(eikon_news_data_final["RIC"])])['RIC'].count().nlargest(10).plot(ax=ax3, kind='bar')
plt.plot()

In [ ]:
#Headlines Size statistics
print('Average word length of Headlines is {0:.0f}.'.format(np.mean(eikon_news_data_final["text"].apply(lambda x: len(x.split())))))
print('Std Dev word length of Headlines is {0:.1f}.'.format(np.std(eikon_news_data_final["text"].apply(lambda x: len(x.split())))))
print('Max word length of Headlines is {0:.0f}.'.format(np.max(eikon_news_data_final["text"].apply(lambda x: len(x.split())))))
print('Average character length of Headlines is {0:.0f}.'.format(np.mean(eikon_news_data_final["text"].apply(lambda x: len(x)))))

In [ ]:
#Headline top words (removing redundant ones)
headline_words = [''.join(filter(str.isalnum, word.lower())) for head in eikon_news_data_final["text"].str.split().values.tolist() for word in head if ''.join(filter(str.isalnum, word.lower()))!=""]
counter=Counter(headline_words)
most=counter.most_common()
stop=set(stopwords.words('english'))
x, y= [], []
for word,count in most[:35]:
    if (word not in stop):
        x.append(word)
        y.append(count)
sns.barplot(x=y,y=x, )
sns.set(rc={'figure.figsize':(11,12)})

* Auxiliary tables to all statistic

In [ ]:
join_leave_by_ric_final = eikon_news_data_final[["RIC","days","leave","join"]].drop_duplicates()

>> **Statistics Saving Setup**

In [ ]:
#SAVE DATA
save_data_file_exploration = input("Create File to Save the Data?")
if save_data_file_exploration =="yes":
    datestring_time = datetime.strftime(datetime.now(),"%m_%d_%Y")
    writer = pd.ExcelWriter("3)Tables_PLots/" + str(datestring_time) + "__Data_Exploration.xlsx", engine='xlsxwriter')
    
save_data_exploration = input("Save the Data?")
if save_data_exploration =="yes":
    writer.save()

* Daily Company Statistics

In [ ]:
    #Aggregate
news_company_level = eikon_news_data_final.groupby(['RIC']).agg(**{'# news':pd.NamedAgg(column='text', aggfunc='count'),
                                                                 '# days with news':pd.NamedAgg(column='Date', aggfunc='nunique')})
news_company_level_DCS = pd.merge(news_company_level, join_leave_by_ric_final[["RIC","days"]], left_index=True, right_on='RIC').set_index('RIC')
    #Main metircs
news_company_level_DCS['avg # news'] = news_company_level_DCS['# news']/news_company_level_DCS['days']
news_company_level_DCS['% days without news'] = (news_company_level_DCS['days']-news_company_level_DCS['# days with news'])/news_company_level_DCS['days']*100
percentiles = [0,10,20,30,40,50,60,70,80,90,100]
new_decile_company_level = news_company_level_DCS.groupby(pd.cut(news_company_level_DCS["avg # news"], np.percentile(news_company_level_DCS["avg # news"], percentiles), include_lowest=True)).agg(
                                                                  **{'avg # news':pd.NamedAgg(column='avg # news', aggfunc='mean'),
                                                                     'median # news':pd.NamedAgg(column='avg # news', aggfunc='median'),
                                                                     '# companies':pd.NamedAgg(column='avg # news', aggfunc='count'),
                                                                      '% days without news':pd.NamedAgg(column='% days without news', aggfunc="mean")})
new_level = news_company_level_DCS.agg(**{'avg # news':pd.NamedAgg(column='avg # news', aggfunc='mean'),
                                                                     'median # news':pd.NamedAgg(column='avg # news', aggfunc='median'),
                                                                     '# companies':pd.NamedAgg(column='avg # news', aggfunc='count'),
                                                                      '% days without news':pd.NamedAgg(column='% days without news', aggfunc="mean")})
display(new_decile_company_level)
display(new_level)

In [ ]:
save_data = input("Save the Data?")
if save_data.lower() == "yes":
    # Position the dataframes in the worksheet.
    new_decile_company_level.to_excel(writer, sheet_name='Daily_Company_Stats')  
    new_level.to_excel(writer, sheet_name='Daily_Company_Stats',startrow=12)

* Extreme Companies statistics (according to total news)

In [ ]:
    #Prepare auxiliary statistics
        #Company level (Total news & auxiliary  # days with news)
news_company_level = eikon_news_data_final.groupby(['RIC']).agg(**{'total # news':pd.NamedAgg(column='text', aggfunc='count'),
                                                                 '# days with news':pd.NamedAgg(column='Date', aggfunc='nunique')})
        #Company Date level (auxiliary list of news count for median, max, min)
news_company_date_level = eikon_news_data_final.groupby(['RIC',"Date"]).agg(**{'text_count':pd.NamedAgg(column='text', aggfunc='count')})
news_company_date_level_countlist = pd.DataFrame(news_company_date_level.groupby('RIC').text_count.apply(list))
news_company_date_level_max_min = news_company_date_level.groupby(['RIC']).agg(**{ 'max # news':pd.NamedAgg(column='text_count', aggfunc='max'),
                                                                                  'min # news':pd.NamedAgg(column='text_count', aggfunc='min')})
    #Aggregate tables
news_ECS_0= pd.merge(news_company_level, join_leave_by_ric_final[["RIC","days"]].drop_duplicates(subset=["RIC"]), how="left", on='RIC').set_index("RIC")
news_ECS_1 = pd.merge(news_ECS_0, news_company_date_level_max_min, left_index=True, right_index=True)
news_ECS = pd.merge(news_ECS_1, news_company_date_level_countlist, left_index=True, right_index=True)
    #Main metircs
news_ECS['avg # news'] = news_ECS['total # news']/news_ECS['days']
news_ECS['% days without news'] = (news_ECS['days']-news_ECS['# days with news'])/news_ECS['days']*100
news_ECS['median # news'] = news_ECS.apply(lambda x: np.median([0]*(x['days']-x['# days with news']) +x['text_count']), axis=1)


    #Select top and bottom companies
top_x_companies = news_ECS.nlargest(10,["total # news"])[['total # news','avg # news','median # news','max # news','min # news','% days without news']]
bottom_x_companies = news_ECS.nsmallest(5,["total # news"])[['total # news','avg # news','median # news','max # news','min # news','% days without news']]

display(top_x_companies)
display(bottom_x_companies)

In [ ]:
save_data = input("Save the Data?")
if save_data.lower() == "yes":
    # Position the dataframes in the worksheet.
    top_x_companies.to_excel(writer, sheet_name='Extreme_Company_Stats')  

* Sector Statistics

In [ ]:
    #Sector names (GICS)
GICS_Sector_lookup = pd.read_excel("1)Eikon_Data/Sectors_lookup__ManualRevision.xlsx")
GICS_Sector_lookup = GICS_Sector_lookup.rename(columns={'Instrument': 'RIC', 'Company Name': 'Company_E(simplified)', 'GICS Sector Name': 'GICS_Sector', 'GICS Industry Group Name': 'GICS_Industry'})
GICS_Sector_lookup = GICS_Sector_lookup[['RIC','Company_E(simplified)','GICS_Sector','GICS_Industry']].copy()
display("Sector Info (GICS)", GICS_Sector_lookup.head(2))

In [ ]:
    #Aggregate
news_company_level = eikon_news_data_final.groupby(['RIC']).agg(**{'# news':pd.NamedAgg(column='text', aggfunc='count'),
                                                                 '# days with news':pd.NamedAgg(column='Date', aggfunc='nunique')})
news_company_level_DCS = pd.merge(news_company_level, join_leave_by_ric_final[["RIC","days"]], left_index=True, right_on='RIC').set_index('RIC')
    #Main metircs
news_company_level_DCS['avg # news'] = news_company_level_DCS['# news']/news_company_level_DCS['days']
news_company_level_DCS['% days without news'] = (news_company_level_DCS['days']-news_company_level_DCS['# days with news'])/news_company_level_DCS['days']*100
news_company_level_Sector = pd.merge(news_company_level_DCS.reset_index(), GICS_Sector_lookup[["RIC","GICS_Sector"]], on='RIC', how="left")
new_decile_sector_level = news_company_level_Sector.groupby(["GICS_Sector"]).agg(
                                                                  **{'avg # news':pd.NamedAgg(column='avg # news', aggfunc='mean'),
                                                                     'median # news':pd.NamedAgg(column='avg # news', aggfunc='median'),
                                                                     '# companies':pd.NamedAgg(column='avg # news', aggfunc='count'),
                                                                      '% days without news':pd.NamedAgg(column='% days without news', aggfunc="mean")})
display(new_decile_sector_level)

In [ ]:
# PFE.N  AMZN.OQ  FB.OQ  TSLA.OQ  MSFT.OQ  AAPL.OQ
#GICS_Sector_lookup[GICS_Sector_lookup["RIC"]=="AAPL.OQ"]

In [ ]:
save_data = input("Save the Data?")
if save_data.lower() == "yes":
    # Position the dataframes in the worksheet.
    new_decile_sector_level.to_excel(writer, sheet_name='Sector_Stats')  

* Source Statistics

In [ ]:
total_news = len(eikon_news_data_final)
new_source_level = eikon_news_data_final.groupby(['sourceCode']).agg(**{'total # news':pd.NamedAgg(column='text', aggfunc='count'),
                                                                        '% news from source': pd.NamedAgg(column='text', aggfunc=lambda x: len(x)/total_news)})
print("Total news", total_news)
display(new_source_level.sort_values(by=['total # news'], ascending=False).head(10))

In [ ]:
save_data = input("Save the Data?")
if save_data.lower() == "yes":
    # Position the dataframes in the worksheet.
    new_source_level.to_excel(writer, sheet_name='Source_Stats')  

# Sentiment Classification

## Functions for generic pre-processing

In [ ]:
def text_processing(text, multiexpression_dic=[], stopwords=None, special_char=False, numbers=True, lowercase=False, stemm=False, lemm=False):
    '''Preprocess a string. All transformation can be chosen through arguments.
    :parameter
    :param text_input: string - sentence/corpus to be processed
    :param numbers: bool - whether numbers are removed or not
    :param special_char: bool - whether special characters and punctuation are removed or not
    :param lowercase: bool - whether  words are converted to lowercase or not   
    :param stopwords: list - list of stopwords to remove
    :param stemm: bool - whether stemming is to be applied
    :param lemm: bool - whether lemmitisation is to be applied
    :param min_size: int - minimum size of words included (inclusive)
    :param tokenize: bool - whether it is to tokenize the final text
    '''
    
    
    # MultiExpression Words (1st step)
    for key, value in multiexpression_dic.items():
        if key in text:
            text = text.replace(key,value)
    # Word Tokenize (2st step)
    text = nltk.word_tokenize(text)    
    #StopWord Removal (3rd step)
    if stopwords is not None:
        text = [word for word in text if word not in stopwords]    
    # Punctuation & Special Character removal (except those between numbers and those in multiexpressions)
    if special_char == True:
        multiexpressions = list(multiexpression_dic.values())
        text = [re.sub(r"(?<!\d)[.,;:](?!\d)", "", word) if word not in multiexpressions else word for word in text] 
        text = [word for word in text if word!=""]   
    # Numbers removal
    if numbers == True:
        text = [re.sub('\d', '#', str(word)) for word in text] 
    # Conversion to lowercase
    if lowercase == True:
        text =  [word.lower() for word in text]       
    # Stemming (remove -ing, -ly, ...)
    if stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        text = [ps.stem(word) for word in text.split()]
    # Lemmatisation (convert the word into root word)
    if lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text.split()]     
    return text

def build_vocab(headlines):
    """REturns dictionary with cout of occurence of each word in the full dataset.
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in headlines:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def embedding_coverage(vocab,model):
    '''Percentage of words in vocabulary that are in model selected. And percentage of tottal text.
    :param vocab: dictionary with count of words in data to be vectorized
    :param model: pretrained model used
    '''
    common_words = {}
    specific_vocab = {}
    n_words_common = 0
    n_specific_vocab = 0
    for word in (vocab):
        try:
            common_words[word] = model[word]
            n_words_common += vocab[word]
        except:
            specific_vocab[word] = vocab[word]
            n_specific_vocab += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(common_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(n_words_common / (n_words_common + n_specific_vocab)))
    sorted_x = sorted(specific_vocab.items(), key=operator.itemgetter(1))[::-1]
    return sorted_x

def FinBERT_tokenization_outputlayer(sentence):
    '''A 3x1 layer is outputed of which the first value represents neutral the second positive and the third negative sentiment  {0:'neutral', 1:'positive',2:'negative'}'''
    try:
        #Tokenizing
        inputs = tokenizer(sentence, return_tensors="pt", padding=True)
        #Output Layes
        outputs = finbert(**inputs)[0]
        array_output = outputs.detach().numpy() 
        return array_output
    except Exception as ex:
        print("Error {} for sentence >  {}".format(ex,sentence))

In [ ]:
def classify_vector(vector, best_model):
    try:
        classification = best_model.predict(vector.reshape(1, -1))[0]
        return classification
    except:
        return np.nan
    
def classify_vector_confidence(vector, best_model):
    try:
        classification = best_model.decision_function(vector.reshape(1, -1))[0]
        return classification
    except:
        return np.nan
    
def classify_vector_probability(vector, best_model):
    try:
        classification = best_model.predict_proba(vector.reshape(1, -1))[0]
        return classification
    except:
        return np.nan

In [ ]:
#Loading pre-trained model and tokenizer
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

## Classification (BEST MODEL)

* Processing (+ finbert)

In [ ]:
#Load Saved Clean Dataframe
SP_clean_0 = pd.read_pickle("1)Eikon_Data/Clean_Eikon_headlines_1Set2020_31Mar2022.pkl") 

#RESAVE Finbert output layer OR load existing
resave_output = input("ReSave Finbert Output?")
if resave_output == "yes":
    SP_clean_0["Finbert_output"] = SP_clean_0.progress_apply(lambda x: FinBERT_tokenization_outputlayer(x["text"]), axis=1)
    with open("4)Eikon_Classification/Processed_Eikon_headlines_1Set2020_31Mar2022.pkl", "wb") as fp:   
        pickle.dump(SP_clean_0, fp)  
else:
    with open("4)Eikon_Classification/Processed_Eikon_headlines_1Set2020_31Mar2022.pkl", "rb") as fp:  
        SP_clean_0 = pickle.load(fp)
print(SP_clean_0.shape)

In [ ]:
#CHECK DUPLICATE OCCURENCES ACROSS COMPANIES TO SEE IF OPPOSING SENTIMENT
perform_dupl_check = input("Perform Check of Repeated News for each Company:  ")
if perform_dupl_check=="yes":
    print("Repetitions of text", set(list(SP_clean_0["Duplicate_Title"])))

    temp_check = SP_clean_0[SP_clean_0["Duplicate_Title"]==2][["text","Company"]]
    temp_check["Companies_Involved"] = temp_check.groupby(["text"])['Company'].transform(lambda x: ', '.join(x))
    temp_check.drop_duplicates(subset=["text","Companies_Involved"], inplace=True)

    for index, row in temp_check.iterrows():
        print(row["Companies_Involved"], ">", row["text"])

* Classify News for Analysis

In [ ]:
#Load best model 
Best_Finbert = load(r"C:\Users\Utilizador\Desktop\MasterDissertation_2021_code\IS_Files\2)Sentiment_Models\RbfSVC_2class_wProb_Finbert.joblib")
reclasify_check = input("Finish Classification of news:  ")
if reclasify_check=="yes":
    # Classify according to binary  (1 for positive and -1 for negative)
    SP_clean_0["Sent_Binary"] = SP_clean_0.progress_apply(lambda x: classify_vector(x["Finbert_output"], Best_Finbert),axis=1)
    # Retrieve probabilities  (vector with probabilty of negative and positive respectively)
    print(f"Classes probability are in the same order as {Best_Finbert.classes_}")
    SP_clean_0["Prob_Vector"] = SP_clean_0.progress_apply(lambda x: classify_vector_probability(x["Finbert_output"], Best_Finbert),axis=1)
    # Use one of the probabilites as Sent, recentering it around 0   (= range from -0.5 to 0.5, with 0 being neutral)
    SP_clean_0["SentProb"] = SP_clean_0.progress_apply(lambda x: x["Prob_Vector"][1]-0.5 ,axis=1)
else:
    with open("4)Eikon_Classification/Classified_Eikon_headlines_1Set2020_31Mar2022", "rb") as fp:  
        SP_clean_0 = pickle.load(fp)
print(SP_clean_0.shape)

In [ ]:
#Safety Check----------------------------------------------
safety_check = input("Run Safety Check? ")
if safety_check == "yes":
    label = {0:-1,1:1}  # position to classification
    safety_check = SP_clean_0[SP_clean_0.progress_apply(lambda x: True if x["Sent_Binary"]==label[np.argmax(x["Prob_Vector"])] else False, axis=1)]
    print(f"Full Dataset is {len(SP_clean_0)}. Safety check keeps {len(safety_check)}")
#-----------------------------------------------------------

In [ ]:
save_output = input("Save classification: ")
if save_output == "yes":
    with open("4)Eikon_Classification/Classified_Eikon_headlines_1Set2020_31Mar2022", "wb") as fp:   
        pickle.dump(SP_clean_0, fp)    

# Daily Complete Data Compilation

Sentiment + Returns + Sector Info + (Auxilairy > Ticker, Cusip)

## Auxiliary Functions

In [ ]:
def news_traiding_daily_timeline_adj(date_value):
    date_time = date_value.replace(tzinfo=None)
    date_ymd = datetime(date_time.year,date_time.month,date_time.day)
    lower_date = datetime(date_ymd.year,date_ymd.month,date_ymd.day,9,30,0)
    upper_date = lower_date + timedelta(hours=23,minutes=30)
    if (date_time>=lower_date) & (date_time<=upper_date):
        return date_ymd
    elif (date_time<= (lower_date - timedelta(minutes=30))):
        return date_ymd - timedelta(days=1)
    else:
        return "Not Considered"

## Compilations Daily

In [ ]:
#Load Saved Dataframe with IntraDay Sentiment classifications
with open("4)Eikon_Classification/Classified_Eikon_headlines_1Set2020_31Mar2022", "rb") as fp:  
    SP_clean_1 = pickle.load(fp)
print(SP_clean_1.shape)

In [ ]:
# Load Returns Data
WRDS_returns_Clean = pd.read_csv("2)WRDS_Data/WRDS_returns_Clean.csv", index_col=0)
WRDS_returns_Clean["DateAdj"] = pd.to_datetime(WRDS_returns_Clean["date"].astype(str))
WRDS_returns = WRDS_returns_Clean[['DateAdj','RIC', 'COMNAM', 'RET', 'sprtrn', 'MktCap']].copy()

* Create Daily Sentiment For Each Company (And Derived Versions)

In [ ]:
# Creating Dates for Daily Aggregation (as explained in paper) > This date contains sentiment actionable in next day
SP_clean_1["DateAdj"] = SP_clean_1.apply(lambda x: news_traiding_daily_timeline_adj(x["DateTime"]), axis=1)
# Remove News between the period of 9:00 AM and 9:30 AM
SP_clean_2 = SP_clean_1[SP_clean_1["DateAdj"]!="Not Considered"].copy()
print(SP_clean_2.shape)

In [ ]:
SentVariableChosen = "AbsoluteRobust"   # "Original"   "RelativeRobust"   "AbsoluteRobust"

> **Original**

In [ ]:
if SentVariableChosen=="Original":
    # Daily Aggregated News
    dict_aggregations = { 'Days':pd.NamedAgg(column='days', aggfunc = lambda x: x.unique()),
                         'Leave':pd.NamedAgg(column='leave', aggfunc = lambda x: x.unique()),
                         'Join':pd.NamedAgg(column='join', aggfunc = lambda x: x.unique()),
                         'Company':pd.NamedAgg(column='Company', aggfunc= lambda x: x.unique()),
                         'DailyTotalSent':pd.NamedAgg(column='SentProb', aggfunc='sum'),  # = overall sentiment score  (using confidence values)
                         'DailyAvgSent':pd.NamedAgg(column='SentProb', aggfunc='mean'),                  
                         'News_vol':pd.NamedAgg(column='SentProb', aggfunc="count")}     # = number of positive news per day 
    # Daily (by RIC) > Note: companies change for some companies so RIC is the most reliable as there was a manual check on values
    SP_Sent_RIC_daily_scores = SP_clean_2.groupby(["RIC","DateAdj"]).agg(**dict_aggregations).reset_index()
    print("Daily sentiment Data", SP_Sent_RIC_daily_scores.shape)

> **Robust***

Relative Measure

In [ ]:
def coerce_values_rel(values):
    sent_aux = (1+np.abs(np.sum(values[values>0])))/(1+np.abs(np.sum(values[values<0])))
    if sent_aux>0:
        return  np.log(sent_aux) 
    else:
        print("Impossible Calc for ", values)
        return np.NaN

In [ ]:
if SentVariableChosen=="RelativeRobust":
    # Daily Aggregated News
    dict_aggregations = { 'Days':pd.NamedAgg(column='days', aggfunc = lambda x: x.unique()),
                         'Leave':pd.NamedAgg(column='leave', aggfunc = lambda x: x.unique()),
                         'Join':pd.NamedAgg(column='join', aggfunc = lambda x: x.unique()),
                         'Company':pd.NamedAgg(column='Company', aggfunc= lambda x: x.unique()),
                         'DailyTotalSent':pd.NamedAgg(column='SentProb', aggfunc='sum'),  # = overall sentiment score  (using confidence values)
                         'DailyAvgSent':pd.NamedAgg(column='SentProb', aggfunc=lambda x: coerce_values_rel(x)),                  
                         'News_vol':pd.NamedAgg(column='SentProb', aggfunc="count")}     # = number of positive news per day 
    # Daily (by RIC) > Note: companies change for some companies so RIC is the most reliable as there was a manual check on values
    SP_Sent_RIC_daily_scores = SP_clean_2.groupby(["RIC","DateAdj"]).agg(**dict_aggregations).reset_index()
    print("Daily sentiment Data", SP_Sent_RIC_daily_scores.shape)

> **Robust****

Absolute Measure

In [ ]:
if SentVariableChosen=="AbsoluteRobust":
    # Daily Aggregated News
    dict_aggregations = { 'Days':pd.NamedAgg(column='days', aggfunc = lambda x: x.unique()),
                         'Leave':pd.NamedAgg(column='leave', aggfunc = lambda x: x.unique()),
                         'Join':pd.NamedAgg(column='join', aggfunc = lambda x: x.unique()),
                         'Company':pd.NamedAgg(column='Company', aggfunc= lambda x: x.unique()),
                         'DailyTotalSent':pd.NamedAgg(column='SentProb', aggfunc='sum'),  # = overall sentiment score  (using confidence values)
                         'DailyAvgSent':pd.NamedAgg(column='SentProb', aggfunc=lambda x: np.abs(np.sum(x[x>0]))-np.abs(np.sum(x[x<0]))),                  
                         'News_vol':pd.NamedAgg(column='SentProb', aggfunc="count")}     # = number of positive news per day 
    # Daily (by RIC) > Note: companies change for some companies so RIC is the most reliable as there was a manual check on values
    SP_Sent_RIC_daily_scores = SP_clean_2.groupby(["RIC","DateAdj"]).agg(**dict_aggregations).reset_index()
    print("Daily sentiment Data", SP_Sent_RIC_daily_scores.shape)

* Organize Sent Data

In [ ]:
#Generate A Dataframe with all date & ticker/RIC Combinations (guarantee the shifted returns & sentiment are correct)
    # Lists of values
complete_date_list = list(WRDS_returns["DateAdj"].unique())
complete_ric_list = list(SP_Sent_RIC_daily_scores["RIC"].unique())
    # Baseline Dataframe
Sent_Headlines =  pd.DataFrame(list(product(complete_date_list, complete_ric_list)), columns=['DateAdj', 'RIC'])
Sent_Headlines = pd.merge(Sent_Headlines,SP_Sent_RIC_daily_scores, how="left", on=['DateAdj', 'RIC'])
    # Sort To ensure Date shift are correctl done
Sent_Headlines = Sent_Headlines.sort_values(["RIC","DateAdj"])
    #Adjust Baseline Dataframe to the timeline for which actual sentiment data exists
Sent_Headlines = Sent_Headlines[(Sent_Headlines["DateAdj"]>=SP_Sent_RIC_daily_scores["DateAdj"].min()) & (Sent_Headlines["DateAdj"]<=SP_Sent_RIC_daily_scores["DateAdj"].max())].copy()  

    #Dealing with Nans
nan_dealing = input("Treat Nans as neutral signals for Delayed Sentiment Value?: ")
if nan_dealing=="yes":
    Sent_Headlines["DailyAvgSent"].fillna(0, inplace=True) 
    
    #Calculate shifted versions (MAIN SENTIMENT MEASURE)
for var in ['DailyAvgSent']:    
    for i in range(1,2):
        Sent_Headlines[var+'(t-' + str(i) +')'] = Sent_Headlines.groupby('RIC')[var].shift(periods=i)
        
Sent_Data_Full = Sent_Headlines.copy()
display(Sent_Data_Full.head(5))

In [ ]:
#Check amount of nans
check_nans = input("Perform Nan check? ")
if check_nans == "yes":
    check_Nans_Sent_Headlines = Sent_Data_Full.copy()
    initial_size=len(check_Nans_Sent_Headlines)
    check_Nans_Sent_Headlines = check_Nans_Sent_Headlines[["DateAdj","RIC"]+[col for col in check_Nans_Sent_Headlines.columns for i in range (1,2 ) if "t-"+str(i) in col]]
    Sent_Headlines_NAN = check_Nans_Sent_Headlines[check_Nans_Sent_Headlines.isnull().sum(axis=1)<3]
    print(f"From {initial_size} observations {len(Sent_Headlines_NAN)/initial_size:.2%} ({len(Sent_Headlines_NAN)}) have reduced number of Nans")
    Sent_Headlines.replace(np.inf, np.nan, inplace=True)
    Sent_Headlines.head(3)

* Auxiliary, Informative Data

In [ ]:
    # GICS Code
GICS_Sectors = pd.read_excel("1)Eikon_Data/Sectors_lookup__ManualRevision.xlsx", index_col=0)
GICS_Sectors.rename(columns = {'RIC Code':'RIC','GICS Sector Name':'GICS_Sector'}, inplace = True)
Sent_GICS = Sent_Data_Full.merge(GICS_Sectors[['RIC', 'GICS_Sector']], how="left", on="RIC")
print("Daily sentiment Data With Ticker and Cusip nd GICS", Sent_GICS.shape)
display(Sent_GICS.head(2))

* Return Data

In [ ]:
#Generate a Dataframe will all date & ticker/RIC Combinations (guarantee the shifted returns & sentiment are correct)
    #Lists of values
complete_date_list = list(WRDS_returns["DateAdj"].unique())
complete_ric_list = list(SP_Sent_RIC_daily_scores["RIC"].unique())
    #Baseline Dataframe
WRDS_returns_Final =  pd.DataFrame(list(product(complete_date_list, complete_ric_list)), columns=['DateAdj', 'RIC'])
WRDS_returns_Final = pd.merge(WRDS_returns_Final,WRDS_returns, how="left", on=['DateAdj', 'RIC'])
WRDS_returns_Final['RET'] = pd.to_numeric(WRDS_returns_Final['RET'], errors='coerce')
WRDS_returns_Final = WRDS_returns_Final.sort_values(["RIC","DateAdj"])

    #Calculate excess returns 
WRDS_returns_Final["Excess_RET"] = WRDS_returns_Final["RET"] - WRDS_returns_Final["sprtrn"]

    #Calculate Shifted market cap for Value weighted analysis
for i in range(1,2):
    WRDS_returns_Final['MktCap(t-' + str(i) +')'] = WRDS_returns_Final.groupby('RIC')['MktCap'].shift(periods=i)
 
    #Eliminate instances were all returns null
WRDS_returns_Final = WRDS_returns_Final[WRDS_returns_Final.isnull().sum(axis=1)<5]
WRDS_returns_Final.head(3)

* Sentiment (and Aux)  +  Return

In [ ]:
print("Size Sent Data", len(Sent_GICS), " | Size Returns", len(WRDS_returns_Final))
DailySentimentReturn = pd.merge(Sent_GICS,WRDS_returns_Final, how="inner", on=['DateAdj', 'RIC'])
print("size Merged Data", len(DailySentimentReturn))
print("----------")
print("Dropped Dates (from Sent)", list(set(Sent_GICS["DateAdj"].unique())-set(DailySentimentReturn["DateAdj"].unique())))
print("Dropped Dates (from Ret)", list(set(WRDS_returns_Final["DateAdj"].unique())-set(DailySentimentReturn["DateAdj"].unique())))
print("----------")
print("Dropped RIC (from Sent)", list(set(Sent_GICS["RIC"].unique())-set(DailySentimentReturn["RIC"].unique())))
print("Dropped RIC (from Ret)", list(set(WRDS_returns_Final["RIC"].unique())-set(DailySentimentReturn["RIC"].unique())))

* Safety Checks

In [ ]:
# Duplicate Check
print( "N of duplicates (on RIC and Date):", len(DailySentimentReturn[DailySentimentReturn.duplicated(subset=['DateAdj','RIC'])]))

In [ ]:
#Check presence of full return timeline for each ric
if False:
    abnormal_values = []
    cal = USFederalHolidayCalendar()
    us_holidays = cal.holidays(start=DailySentimentReturn["DateAdj"].min(), end=DailySentimentReturn["DateAdj"].max()).to_pydatetime()
    us_holidays = np.array([x.date() for x in us_holidays], dtype='datetime64[D]')
    for ric_aux in list(DailySentimentReturn["RIC"].unique()):
        temp = DailySentimentReturn[DailySentimentReturn["RIC"]==ric_aux].copy()
        n_business_days = np.busday_count(temp["DateAdj"].min().date(), temp["DateAdj"].max().date(), holidays=us_holidays)
        unique_days = temp["DateAdj"].nunique()
        days = len(temp["DateAdj"])
        if not (n_business_days==unique_days==days):
            abnormal_values.append({"ric":ric_aux, "Bdays":n_business_days, "Adays": [unique_days,days]})
    print(abnormal_values)        
    #Further check missing dates ......

In [ ]:
#Check Abnormal Values
Infinite_check = DailySentimentReturn[DailySentimentReturn.isin([np.inf, -np.inf]).any(axis=1)]
print(f"From {len(DailySentimentReturn)} observations {len(Infinite_check)/len(DailySentimentReturn):.2%} ({len(Infinite_check)}) have infinite values")
print("Infinite Values are present in", Infinite_check.columns[Infinite_check.isin([np.inf, -np.inf]).any(axis=0)])

force_inf = input("Convert Infinite Values to Nan? ")
if force_inf=="yes":
    DailySentimentReturn.replace(np.inf, np.nan, inplace=True)
    DailySentimentReturn.replace(-np.inf, np.nan, inplace=True)

In [ ]:
#Nan Checks
Nan_check = DailySentimentReturn[["DateAdj","RIC"]+[col for col in DailySentimentReturn.columns for i in range (1,2 ) if "t-"+str(i) in col]]
Sent_Headlines_NAN = Nan_check[Nan_check.isnull().sum(axis=1)<3]
print(f"From {len(DailySentimentReturn)} observations {len(Sent_Headlines_NAN)/len(DailySentimentReturn):.2%} ({len(Sent_Headlines_NAN)}) have reduced number of Nans")

* Save Data

In [ ]:
save_output = input("Save Complete Daily Data: ")

if save_output == "yes":
    if SentVariableChosen=="Original":
        with open("4)Eikon_Classification/Daily_Complete_HeadlineReturn", "wb") as fp:   
            pickle.dump(DailySentimentReturn, fp)  
    elif SentVariableChosen=="RelativeRobust":    
        with open("4)Eikon_Classification/Daily_Complete_HeadlineReturn_RelativeRobust", "wb") as fp:   
            pickle.dump(DailySentimentReturn, fp)  
    elif SentVariableChosen=="AbsoluteRobust":  
        with open("4)Eikon_Classification/Daily_Complete_HeadlineReturn_AbsoluteRobust", "wb") as fp:   
            pickle.dump(DailySentimentReturn, fp)  
    print("Saved File for Variable", SentVariableChosen)

## Compilations Weekly

In [ ]:
#Load Saved Dataframe with IntraDay Sentiment classifications
with open("4)Eikon_Classification/Classified_Eikon_headlines_1Set2020_31Mar2022", "rb") as fp:  
    SP_clean_1 = pickle.load(fp)

In [ ]:
# Load Returns Data
WRDS_returns_Clean = pd.read_csv("2)WRDS_Data/WRDS_returns_Clean.csv", index_col=0)
WRDS_returns_Clean["DateAdj"] = pd.to_datetime(WRDS_returns_Clean["date"].astype(str))
WRDS_returns = WRDS_returns_Clean[['DateAdj','RIC', 'COMNAM', 'RET', 'sprtrn', 'MktCap']].copy()

* Create Weekly Sentiment For Each Company (And Derived Versions)

In [ ]:
#Dealing with Nans
nan_dealing_extra = input("Treat Nans as neutral signals for Extra Variables based on Tendencies?: ")
    
#Calculate Sent Schock Score (attributing a score just in the cases for which at least half of the lookback period is available)
lookback = 7
final_temp = pd.DataFrame()
    #(create for each RIC a list with the necesssary lookback period for calculations)
for tic in Sent_Headlines['RIC'].unique():
    temp = Sent_Headlines[Sent_Headlines['RIC']==tic].copy()
    temp.sort_values(by=['DateAdj'], inplace=True)
    if nan_dealing_extra == "yes":  
        temp['List_SentShock(t-1)'] = [[0 if math.isnan(value) else value for value in window.to_list()] for window in temp["DailyAvgSent(t-1)"].rolling(window=lookback)]
    else:
        temp['List_SentShock(t-1)'] = [window.to_list() for window in temp["DailyAvgSent(t-1)"].rolling(window=lookback)]
    final_temp = pd.concat([final_temp,temp])
    #(auxiliary variables)
final_temp["Average("+str(lookback)+" days)"] = final_temp.apply(lambda x : np.nanmean(x['List_SentShock(t-1)']), axis=1)
final_temp["Std("+str(lookback)+" days)"] = final_temp.apply(lambda x : np.std(x['List_SentShock(t-1)']), axis=1)
final_temp["Count("+str(lookback)+" days)"] = final_temp.apply(lambda x : np.sum(~np.isnan(x['List_SentShock(t-1)'])) if type(x['List_SentShock(t-1)'])==list else 0, axis=1)
    #(attributing a score just in the cases for which at least 1/3 of the lookback period is available)
final_temp["Sent_Shock_"+str(lookback)+"(t)"] = np.where(final_temp["Count("+str(lookback)+" days)"]>lookback/3,(final_temp["DailyAvgSent"] - final_temp["Average("+str(lookback)+" days)"])/final_temp["Std("+str(lookback)+" days)"], np.nan)
final_temp["Sent_Shock_"+str(lookback)+"(t-1)"] = final_temp.groupby('RIC')["Sent_Shock_"+str(lookback)+"(t)"].shift(periods=1)
    #(show key info)
count_valid_observations = np.sum(~np.isnan(final_temp["Sent_Shock_"+str(lookback)+"(t-1)"]))
print(f"From {len(final_temp)} observations {count_valid_observations/len(final_temp):.2%} ({count_valid_observations}) have valid Sentiment schock scores")

#Calculate Sent Trend score
lookback = 7
    #(sentiment delta)
if nan_dealing_extra == "yes":
    final_temp["ΔSent(t)"] = final_temp["DailyAvgSent"].fillna(0) - final_temp["DailyAvgSent(t-1)"].fillna(0)
else:
    final_temp["ΔSent(t)"] = final_temp["DailyAvgSent"] - final_temp["DailyAvgSent(t-1)"]
final_temp_2 = pd.DataFrame()
    #(create for each RIC a list with the necesssary lookvack period fro calculations)
for tic in final_temp['RIC'].unique():
    temp = final_temp[final_temp['RIC']==tic].copy()
    temp.sort_values(by=['DateAdj'], inplace=True)
    temp['List_ΔSent(t)'] = [window.to_list() for window in temp["ΔSent(t)"].rolling(window=lookback)]
    final_temp_2 = pd.concat([final_temp_2,temp])
    #(auxiliary variables)
final_temp_2["Sum_Trend("+str(lookback)+" days)"] = final_temp_2.apply(lambda x : np.sum(x['List_ΔSent(t)']), axis=1)
final_temp_2["Count_Trend("+str(lookback)+" days)"] = final_temp_2.apply(lambda x : np.sum(~np.isnan(x['List_ΔSent(t)'])) if type(x['List_ΔSent(t)'])==list else 0, axis=1)
    #(attributing a score just in the cases for which at least 1/3 of the lookback period is available)
final_temp_2["Sent_Trend_"+str(lookback)+"(t)"] = np.where(final_temp_2["Count_Trend("+str(lookback)+" days)"]>lookback/3, final_temp_2["Sum_Trend("+str(lookback)+" days)"] , np.nan)
final_temp_2["Sent_Trend_"+str(lookback)+"(t-1)"] = final_temp_2["Sent_Trend_"+str(lookback)+"(t)"].shift(periods=1)
    #(show key info)
count_valid_observations = np.sum(~np.isnan(final_temp_2["Sent_Trend_"+str(lookback)+"(t-1)"]))
print(f"From {len(final_temp_2)} observations {count_valid_observations/len(final_temp_2):.2%} ({count_valid_observations}) have valid Sentiment Trend scores")

#Save in Variable
Sent_Data_Full = final_temp_2.copy()
display(Sent_Data_Full.head(5))

In [ ]:
#SAme as other but filter in the end for the signals on monday (exclusive) so that portfolio can be adjusted in that day

# Data Exploration (OS Data)

## Summary Statistics (Sentiment by News)

In [ ]:
daily_input = input("Use daily data?")

if daily_input!="yes":
    #Load Saved Dataframe with IntraDay Sentiment classifications
    with open("4)Eikon_Classification/Classified_Eikon_headlines_1Set2020_31Mar2022", "rb") as fp:  
        Os_Data_Explore = pickle.load(fp)
    print(Os_Data_Explore.shape)
else:
    #Load Saved Dataframe with Daily Sentiment classifications
    with open("4)Eikon_Classification/Daily_Complete_HeadlineReturn", "rb") as fp:  
        Os_Data_Explore = pickle.load(fp)
        Os_Data_Explore["SentProb"] = Os_Data_Explore["DailyAvgSent"]
    print(Os_Data_Explore.shape)

* Distribution

In [ ]:
print("Overall Distribution")
display(Os_Data_Explore["SentProb"].describe())
display("Kurtosis:", Os_Data_Explore["SentProb"].kurtosis())
display("Skewness:", Os_Data_Explore["SentProb"].skew())
#Displays
fig, axes = plt.subplots(nrows=1,ncols=2)
fig.set_size_inches(18, 10, forward=True)
display(Os_Data_Explore["SentProb"].hist(bins=50, ax= axes[0]))
ax1 = plt.gca()
display(Os_Data_Explore.boxplot(column="SentProb", ax= axes[1]))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'figure.figsize':(10,5), 'figure.dpi':100})
fig, ax = plt.subplots()
# Hide the top and right spines of the axis
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
#plt.xticks(np.arange(-6, 9, 1.0))

# Plot Histogram on x
x = Os_Data_Explore["SentProb"]
plt.hist(x, bins=40)
plt.gca().set(#title='Histogram', 
              ylabel='News Volume',xlabel="Sentiment")
#Save
if save_data_file_exploration.lower() == "yes":
    plt.savefig('3)Tables_PLots/Sentiment Distribution.png',dpi=300, transparent=False, bbox_inches='tight')

* Headline Example

In [ ]:
company = "AMZN.OQ"
company_table = Os_Data_Explore[Os_Data_Explore["RIC"]== company]
company_extreme_news = pd.concat([pd.DataFrame(company_table.nlargest(10,"SentProb")),
                          pd.DataFrame(company_table.nsmallest(20,"SentProb"))])

company_table[company_table.columns] = company_table[company_table.columns].astype(str)
company_extreme_news[company_extreme_news.columns] = company_extreme_news[company_extreme_news.columns].astype(str) 
company_extreme_news.to_excel("3)Tables_Plots/SentimentExplore"+ company + "_Headlines.xlsx")
company_table.to_excel("3)Tables_Plots/SentimentExploreFull"+ company + "_Headlines.xlsx")
company_table.head(2)

>> **Statistics Saving Setup**

In [ ]:
#SAVE DATA
save_data_file_exploration = input("Create File to Save the Data?")
if save_data_file_exploration =="yes":
    datestring_time = datetime.strftime(datetime.now(),"%m_%d_%Y")
    writer_sent = pd.ExcelWriter("3)Tables_PLots/" + str(datestring_time) + "__Data_ExplorationSENT.xlsx", engine='xlsxwriter')
    
save_data_exploration = input("Save the Data?")
if save_data_exploration =="yes":
    writer_sent.save()

In [ ]:
#Load Saved Dataframe with IntraDay Sentiment classifications
#  Daily_Complete_HeadlineReturn
#  Daily_Complete_HeadlineReturn_AbsoluteRobust
#  Daily_Complete_HeadlineReturn_RelativeRobust
with open("4)Eikon_Classification/Daily_Complete_HeadlineReturn_RelativeRobust", "rb") as fp:  
    InvestData_Explore = pickle.load(fp)
print(InvestData_Explore.shape)

* Plot Graph Sent versus returns

In [ ]:
#columns list 
col_used = ['DailyAvgSent(t-1)','RET',"MktCap(t-1)",'sprtrn']

# data
    #frequency adjustment
InvestData_Explore_W["frequency"] = InvestData_Explore_W['DateAdj'].dt.strftime('%Y-%U')   # WEEK
#InvestData_Explore_W["frequency"] = InvestData_Explore_W['DateAdj']   # DAY
news_day_level = InvestData_Explore_W.groupby(['frequency',"RIC"]).mean().reset_index()
    #separate by section
daily_top_mkt = news_day_level.groupby(['frequency']).apply(lambda x: x[(x["MktCap(t-1)"]>=np.percentile(x["MktCap(t-1)"],95))])
daily_top_mkt = daily_top_mkt[~daily_top_mkt.isnull()[col_used].any(axis=1)].copy()
daily_bottom_mkt = news_day_level.groupby(['frequency']).apply(lambda x: x[(x["MktCap(t-1)"]<=np.percentile(x["MktCap(t-1)"],5))])
daily_bottom_mkt = daily_bottom_mkt[~daily_bottom_mkt.isnull()[col_used].any(axis=1)].copy()

# plot pos and negative in differnt colors
plt.scatter(daily_top_mkt["DailyAvgSent(t-1)"], daily_top_mkt["RET"], color='green', label="Pos Sent", alpha=0.2) #pos/top
plt.scatter(daily_bottom_mkt["DailyAvgSent(t-1)"], daily_bottom_mkt["RET"], color='red', label="Neg Sent", alpha=0.2)  #neg/bottom
#settings
plt.legend()

* Plot with Sent, return and Volume

In [ ]:
    #Aggregate
InvestData_Explore_W =  InvestData_Explore.copy() 
InvestData_Explore_W["frequency"] = InvestData_Explore_W['DateAdj'].dt.strftime('%Y-%U')   # WEEK
#InvestData_Explore_W["frequency"] = InvestData_Explore_W['DateAdj']   # DAY
news_day_level = InvestData_Explore_W.groupby(['frequency']).agg(**{'avgSent':pd.NamedAgg(column='DailyAvgSent', aggfunc=lambda x: x.mean()),
                                                                'volume':pd.NamedAgg(column='News_vol', aggfunc=lambda x: x.sum()),
                                                                's&p':pd.NamedAgg(column='sprtrn', aggfunc=lambda x: x.mean()),
                                                                'date':pd.NamedAgg(column='DateAdj', aggfunc=lambda x: x.max())})

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
ax2 = ax.twinx()  # set up the 2nd axis
ax3 = ax.twinx() # set up 3rd axis
alphaVal = 1
linethick=2

#sentiment
lg1= ax.plot(news_day_level.date,
        news_day_level["avgSent"],
        color="#00008b",
        alpha=0.7,
        linestyle = '-',
        lw=linethick,
        label="Average Sentiment",
       # alpha=alphaVal
       )
# Return S&P
lg2= ax2.plot(news_day_level.date,
        news_day_level["s&p"],
        color="green",
        linestyle = '-',
        alpha=0.7,
        lw=linethick,
        label="S&P 500 Index",
       # alpha=alphaVal
       )
# Volume
ax3.bar(news_day_level.date,
        news_day_level["volume"],
        color="grey",
        alpha=0.2, 
        width=5,
        #color="blue",
        label="News Vol.",
        #alpha=alphaVal
       )



# Set Axis
ax3.get_yaxis().set_visible(False)
myFmt = matplotlib.dates.DateFormatter('%Y-%m')
ax.xaxis.set_major_formatter(myFmt)

#Anotate Volume extreme value
    #(max)
y_max = news_day_level["volume"].max()
x_max = news_day_level[news_day_level["volume"]==news_day_level["volume"].max()]["date"].values[0]
ax3.annotate("#News={0:,.0f}".format(y_max), xy=(x_max, y_max),bbox=dict(pad=5, facecolor="none", edgecolor="none"))
    #(min)
y_min = news_day_level["volume"].min()
x_min = news_day_level[news_day_level["volume"]==news_day_level["volume"].min()]["date"].values[0]
ax3.annotate("#News={0:,.0f}".format(y_min), xy=(x_min, y_min),bbox=dict(pad=5, facecolor="none", edgecolor="none"))

# Set Labels
ax.set_xlabel('')
ax.set_xlabel("Month",rotation=0)
#plt.ylabel("Average Sentiment Confidence",rotation=90)
ax.set_ylabel('Average Sentiment')
ax2.set_ylabel('Average S&P 500 Index Return')

#Legend
lns = lg1+lg2
labs = [l.get_label() for l in lg1+lg2]
ax.legend(lns, labs, frameon=False,  loc='lower left',ncol=1,handlelength=1,framealpha=1,facecolor="w")

#Save
if save_data_file_exploration.lower() == "yes":
    plt.savefig('3)Tables_PLots/Weekly_Sent_Return_Volume.png')

* Sent by Decile (average news)

In [ ]:
    #Aggregate
news_company_level = InvestData_Explore.groupby(['RIC']).agg(**{'avgSent':pd.NamedAgg(column='DailyAvgSent', aggfunc=lambda x: x.mean()),
                                                                'news_vol':pd.NamedAgg(column='News_vol', aggfunc=lambda x: x.sum()),
                                                                 'days':pd.NamedAgg(column='Days', aggfunc=lambda x: x.max())})
news_company_level['avg # news'] = news_company_level['news_vol']/news_company_level['days']
    #Main metircs
percentiles = [0,10,20,30,40,50,60,70,80,90,100]
new_decile_company_level = news_company_level.groupby(pd.cut(news_company_level["avg # news"], np.nanpercentile(news_company_level["avg # news"], percentiles), include_lowest=True)).agg(
                                                                  **{'avg Sent':pd.NamedAgg(column='avgSent', aggfunc='mean'),
                                                                     'median Sent':pd.NamedAgg(column='avgSent', aggfunc='median'),
                                                                     'max Sent':pd.NamedAgg(column='avgSent', aggfunc='max'),
                                                                     'min Sent':pd.NamedAgg(column='avgSent', aggfunc='min'),
                                                                     'ske Sent':pd.NamedAgg(column='avgSent', aggfunc='skew'),
                                                                     'kurt Sent':pd.NamedAgg(column='avgSent', aggfunc='min'),
                                                                    })
display(new_decile_company_level)

In [ ]:
# Position the dataframes in the worksheet.
if save_data_file_exploration =="yes":
    new_decile_company_level.reset_index().to_excel(writer_sent, sheet_name='Decile_Stats')  

* Sent by Sector(average sent)

In [ ]:
GICS_Sector_lookup = pd.read_excel("1)Eikon_Data/Sectors_lookup__ManualRevision.xlsx", index_col=0)
GICS_Sector_lookup.rename(columns = {'RIC Code':'RIC','GICS Sector Name':'GICS_Sector'}, inplace = True)

In [ ]:
# BY SECTOR
    #Aggregate
news_company_level = InvestData_Explore.groupby(['RIC']).agg(**{'avgSent':pd.NamedAgg(column='DailyAvgSent', aggfunc=lambda x: x.mean())})
    #Main metircs
news_company_level_Sector = pd.merge(news_company_level.reset_index(), GICS_Sector_lookup[["RIC","GICS_Sector"]], on='RIC', how="left")
new_decile_sector_level = news_company_level_Sector.groupby(["GICS_Sector"]).agg(
                                                                  **{'avg Sent':pd.NamedAgg(column='avgSent', aggfunc='mean'),
                                                                     'median Sent':pd.NamedAgg(column='avgSent', aggfunc='median'),
                                                                     'max Sent':pd.NamedAgg(column='avgSent', aggfunc='max'),
                                                                     'min Sent':pd.NamedAgg(column='avgSent', aggfunc='min'),
                                                                     'ske Sent':pd.NamedAgg(column='avgSent', aggfunc='skew'),
                                                                     'kurt Sent':pd.NamedAgg(column='avgSent', aggfunc='min'),
                                                                    })
display(new_decile_sector_level)

In [ ]:
# Position the dataframes in the worksheet.
if save_data_file_exploration =="yes":
    new_decile_sector_level.reset_index().to_excel(writer_sent, sheet_name='Sector_Stats')  

# Investment Strats  (daily)

Fama/French 3 Factor >  https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

## Auxiliary Functions

### Stats

MAX Drawdaown
https://stackoverflow.com/questions/52539335/how-to-calculate-maximumdrawdown-using-returns-in-python

Turnover
https://quant.stackexchange.com/questions/19229/calculate-turnover-for-portfolio

In [ ]:
#Annualized results
def advanced_stats_table(Strat_Returns):
    #Main Metrics
    Strat_Statistics.loc["Count"] = len(Strat_Returns)
    Strat_Statistics.loc[""] = ""
    Strat_Statistics.loc["Mean %"] = np.average(Strat_Returns)*100*12
    Strat_Statistics.loc["(mean) t-stat"] = np.average(Strat_Returns) / (np.std(Strat_Returns)/ (Strat_Statistics.loc["Count"].values[0]**0.5)) 
    Strat_Statistics.loc["(mean) p-value"] = (1-norm.cdf(Strat_Statistics.loc["(mean) t-stat"].values[0]))*2
    Strat_Statistics.loc["-"] = "-"
    Strat_Statistics.loc["Std. Dev. (%)"] = np.round(np.std(Strat_Returns)*100*(12**0.5),2)
    Strat_Statistics.loc["--"] = "--"
    Strat_Statistics.loc["Sharpe ratio"] = Strat_Statistics.loc["Mean %"]/ Strat_Statistics.loc["Std. Dev. (%)"]
    Strat_Statistics.loc["(SR) t-stat"] = (Strat_Statistics.loc["Sharpe ratio"].values[0]/(12**0.5))/np.sqrt((1+0.5*(Strat_Statistics.loc["Sharpe ratio"].values[0]/(12**0.5))**2)/Strat_Statistics.loc["Count"].values[0])
    Strat_Statistics.loc["(SR) p-value"] = (1-norm.cdf(Strat_Statistics.loc["(SR) t-stat"].values[0]))*2
    Strat_Statistics.loc["---"] = "---"
    Strat_Statistics.loc["Skewness"] = skew(Strat_Returns)
    Strat_Statistics.loc["Kurtosis"] = kurtosis(Strat_Returns)
    Strat_Statistics.loc["----"] = "----"
    Strat_Statistics.loc["JB test statistic"] = (Strat_Statistics.loc["Count"].values[0]/6)*(Strat_Statistics.loc["Skewness"].values[0]**2+0.25*(Strat_Statistics.loc["Kurtosis"].values[0]**0.5))
    Strat_Statistics.loc["(JB) p-value"] = 1-chi2.cdf(Strat_Statistics.loc["JB test statistic"].values[0], 2)
    Strat_Statistics.loc["-----"] = "-----"
    Strat_Statistics.loc["Minimum (%)"] = Strat_Returns.min()
    Strat_Statistics.loc["Percentile 25 (%)"] = np.percentile(Strat_Returns, 25)
    Strat_Statistics.loc["Median (%)"] = Strat_Returns.median()
    Strat_Statistics.loc["Percentile 75 (%)"] = np.percentile(Strat_Returns, 75)
    Strat_Statistics.loc["Maximum (%)"] = Strat_Returns.max()
    Strat_Statistics.loc["------"] = "------"
    Strat_Statistics.loc["AR(1) (%)"] = Strat_Returns.corr(Strat_Returns.shift(1))
    Strat_Statistics.loc["p-value"] = 2*(1-norm.cdf(Strat_Statistics.loc["Count"].values[0]**0.5*np.abs(Strat_Statistics.loc["AR(1) (%)"].values[0])))
    return Strat_Statistics

In [ ]:
def paper_stats_table(STRAT, ret_col,cumret_col, data):
    #(auxiliary)
    Strat_Statistics = pd.DataFrame()
    ret = data[ret_col]
    cum_ret = data[cumret_col]
    #(average)
    Strat_Statistics.loc[STRAT, "Average Return (%)"] = np.average(ret)*100*12
    Strat_Statistics.loc["(mean) t-stat", "Average Return (%)"] = np.average(ret) / (np.std(ret)/ (len(ret)**0.5)) 
    Strat_Statistics.loc["(mean) p-value", "Average Return (%)"] = (1-norm.cdf(np.abs(Strat_Statistics.loc["(mean) t-stat", "Average Return (%)"])))*2
    #(sharpe ratio)
    Strat_Statistics.loc[STRAT, "Sharpe Ratio"] = (np.average(ret)*12)/ (np.std(ret)*(12**0.5)) 
    Strat_Statistics.loc["(mean) t-stat", "Sharpe Ratio"] = (Strat_Statistics.loc[STRAT, "Sharpe Ratio"]/(12**0.5))/np.sqrt((1+0.5*(Strat_Statistics.loc[STRAT, "Sharpe Ratio"]/(12**0.5))**2)/len(ret))
    Strat_Statistics.loc["(mean) p-value", "Sharpe Ratio"] = (1-norm.cdf(np.abs(Strat_Statistics.loc["(mean) t-stat", "Sharpe Ratio"])))*2
    #(max drawdown)
    aux_drawdown = pd.DataFrame(ret*1000).rename(columns={ret_col:"Dollar"})
    aux_drawdown["Dollar"].iloc[0] += 1000
    aux_drawdown["DollarEvol"] = aux_drawdown["Dollar"].cumsum()
    aux_drawdown["Roll_Max"] = aux_drawdown['DollarEvol'].rolling(300, min_periods=1).max()
    aux_drawdown["Daily_Drawdown"] = aux_drawdown["DollarEvol"]/aux_drawdown["Roll_Max"] - 1.0
    Strat_Statistics.loc[STRAT, "Max Drawdown (%)"] = (aux_drawdown["Daily_Drawdown"].min()) *100
    #(3 Factor)
    X_Fama3Factors = data[['Mkt-RF', 'SMB', 'HML']]
    Y_ExcessReturns = data[ret_col] - data['RF']
    X = sm.add_constant(X_Fama3Factors)
    ff_model = sm.OLS(Y_ExcessReturns, X).fit()
    Strat_Statistics.loc["(mean) t-stat", "FF3"] = ff_model.tvalues[0]
    Strat_Statistics.loc["(mean) p-value", "FF3"] = (1-norm.cdf(np.abs(Strat_Statistics.loc["(mean) t-stat", "FF3"])))
    Strat_Statistics.loc["alpha", "FF3"] = ff_model.params[0] * 12 * 100
    Strat_Statistics.loc["R^2", "FF3"] = ff_model.rsquared * 100
    #(5 Factor)
    X_Fama5Factors = data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
    Y_ExcessReturns = data[ret_col] - data['RF']
    X = sm.add_constant(X_Fama5Factors)
    ff_model = sm.OLS(Y_ExcessReturns, X).fit()
    Strat_Statistics.loc["(mean) t-stat", "FF5"] = ff_model.tvalues[0]
    Strat_Statistics.loc["(mean) p-value", "FF5"] = (1-norm.cdf(np.abs(Strat_Statistics.loc["(mean) t-stat", "FF5"])))
    Strat_Statistics.loc["alpha", "FF5"] = ff_model.params[0] * 12 * 100
    Strat_Statistics.loc["R^2", "FF5"] = ff_model.rsquared * 100
    #(5 Factor+MOM)
    X_Fama5FactorsMOM = data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom']]
    Y_ExcessReturns = data[ret_col] - data['RF']
    X = sm.add_constant(X_Fama5FactorsMOM)
    ff_model = sm.OLS(Y_ExcessReturns, X).fit()
    Strat_Statistics.loc["(mean) t-stat", "FF5 + MOM"] = ff_model.tvalues[0]
    Strat_Statistics.loc["(mean) p-value", "FF5 + MOM"] = (1-norm.cdf(np.abs(Strat_Statistics.loc["(mean) t-stat", "FF5 + MOM"])))
    Strat_Statistics.loc["alpha", "FF5 + MOM"] = ff_model.params[0] * 12 * 100
    Strat_Statistics.loc["R^2", "FF5 + MOM"] = ff_model.rsquared * 100
    return Strat_Statistics

In [ ]:
def paper_stats_table_Costs(STRAT, ret_col,net_col, data):
    #(auxiliary)
    Strat_Statistics = pd.DataFrame()
    ret = data[ret_col]
    net_ret = data[net_col]
    #GROSS
        #(average)
    Strat_Statistics.loc[STRAT, "GROSS Return (%)"] = np.average(ret)*100*12
    Strat_Statistics.loc["(mean) t-stat", "GROSS Return (%)"] = np.average(ret) / (np.std(ret)/ (len(ret)**0.5)) 
    Strat_Statistics.loc["(mean) p-value", "GROSS Return (%)"] = (1-norm.cdf(np.abs(Strat_Statistics.loc["(mean) t-stat", "GROSS Return (%)"])))*2
        #(sharpe ratio)
    Strat_Statistics.loc[STRAT, "GROSS SR"] = (np.average(ret)*12)/ (np.std(ret)*(12**0.5)) 
    Strat_Statistics.loc["(mean) t-stat", "GROSS SR"] = (Strat_Statistics.loc[STRAT, "GROSS SR"]/(12**0.5))/np.sqrt((1+0.5*(Strat_Statistics.loc[STRAT, "GROSS SR"]/(12**0.5))**2)/len(ret))
    Strat_Statistics.loc["(mean) p-value", "GROSS SR"] = (1-norm.cdf(np.abs(Strat_Statistics.loc["(mean) t-stat", "GROSS SR"])))*2
    #NET
        #(average)
    Strat_Statistics.loc[STRAT, "NET Return (%)"] = np.average(net_ret)*100*12
    Strat_Statistics.loc["(mean) t-stat", "NET Return (%)"] = np.average(net_ret) / (np.std(net_ret)/ (len(net_ret)**0.5)) 
    Strat_Statistics.loc["(mean) p-value", "NET Return (%)"] = (1-norm.cdf(np.abs(Strat_Statistics.loc["(mean) t-stat", "NET Return (%)"])))*2
        #(sharpe ratio)
    Strat_Statistics.loc[STRAT, "NET SR"] = (np.average(net_ret)*12)/ (np.std(net_ret)*(12**0.5)) 
    Strat_Statistics.loc["(mean) t-stat", "NET SR"] = (Strat_Statistics.loc[STRAT, "NET SR"]/(12**0.5))/np.sqrt((1+0.5*(Strat_Statistics.loc[STRAT, "NET SR"]/(12**0.5))**2)/len(net_ret))
    Strat_Statistics.loc["(mean) p-value", "NET SR"] = (1-norm.cdf(np.abs(Strat_Statistics.loc["(mean) t-stat", "NET SR"])))*2
    return Strat_Statistics

In [ ]:
def FamaFrench(data, y_var):
    X_Fama3Factors = data[['Mkt-RF', 'SMB', 'HML']]
    Y_ExcessReturns = data[y_var] - data['RF']
    X = sm.add_constant(X_Fama3Factors)
    ff_model = sm.OLS(Y_ExcessReturns, X).fit()
    print(ff_model.summary())
    intercept, b1, b2, b3 = ff_model.params
    return intercept, b1, b2, b3

### Portfolio Creation

In [ ]:
def get_ric_pos_dict(strat_,   Ncomp,   date_, row_sent,   df_check):
    '''Dictionary of RIC and respective position 1 (long) or -1 (short), Corrected for Data Availability'''
    #Create dictionary with RIC : Position Sig basedon Strat
    if strat_=="L":
        ric_pos_dict =  {r: 1 for r in list(row_sent[row_sent>0].nlargest(Ncomp).index)}
    elif strat_=="S":
        ric_pos_dict = {r: -1 for r in list(row_sent[row_sent<0].nsmallest(Ncomp).index)}
    elif strat_=="L-S":
        ric_pos_dict = {** {r: 1 for r in list(row_sent[row_sent>0].nlargest(int(Ncomp/2)).index)} ,  **{r: -1 for r in list(row_sent[row_sent<0].nsmallest(int(Ncomp/2)).index)}}
    #Clean list based on data availability
    for df in df_check:
        date_aux_df = df.loc[date_]
        non_nan_rics = list(date_aux_df[~date_aux_df.isna()].index)
        ric_pos_dict = {key:value  for key, value in ric_pos_dict.items() if key in non_nan_rics}
    #Determine what to return 
    if ric_pos_dict!=[]:
        return ric_pos_dict
    else:
        return None

In [ ]:
def get_ric_value_dict(date_, C_ric_pos_dict, Value_df):
    '''Get the list of values (can be Mkt, RET,... depends on the Value_df chosen) for each RIC preserving order of dict'''
    day_values = Value_df.loc[date_]
    sorted_values = [day_values.loc[ric] for ric in C_ric_pos_dict]
    return sorted_values

In [ ]:
def get_universe_weight_dict(unique_rics, C_ric_pos_dict, C_weights=None):
    '''Dictionary of all RIC with the respective positions for that point in time, either equal weighted or value weighted'''
    if C_weights is None:
        size_pos = len(C_ric_pos_dict)
        universe_weights = {ric:(C_ric_pos_dict[ric]/size_pos if ric in C_ric_pos_dict else 0) for ric in unique_rics}
    else:
        size_pos = np.sum(C_weights)
        C_weights_dict = {ric:C_weights[index] for index, ric in enumerate(C_ric_pos_dict)}
        universe_weights = {ric:(C_ric_pos_dict[ric]*C_weights_dict[ric]/size_pos if ric in C_ric_pos_dict else 0) for ric in unique_rics}
    return universe_weights

In [ ]:
def get_daily_turnover(date_, weights_df, Return_df,tipo="adj"):
    '''Based on date get current and previous wight lists for the all the RICs and compute abs difference 
    adjusted based on returns to get turnover,  the first position is assumed to be zero on all rics'''
    #Weight Universes
    loc_row = weights_df.index.get_loc(date_)
    C_weights = weights_df.iloc[loc_row]
    P_weights = weights_df.iloc[loc_row-1] if loc_row>0 else {key:0 for key in C_weights}
    #Return Universes
    day_values = Return_df.loc[date_]
    C_returns = {key:(0 if pd.isnull(day_values.loc[key]) else day_values.loc[key]) for key in C_weights}
    return_if_hold = np.sum([P_weights[key]*C_returns[key] for key in C_returns])
    #Adjusted Calc
    if tipo=="adj":
        cost_list = [np.abs(C_weights[key]-(P_weights[key]*(1+C_returns[key]))/(1+return_if_hold)) for key in C_weights]
        return np.sum(cost_list)/2
    else:
        cost_list = [np.abs(C_weights[key]-P_weights[key]) for key in C_weights]
        return np.sum(cost_list)/2        

In [ ]:
def get_transaction_costs(date_, weight_df):
    '''Based on date get current and previous wight lists for the all the RICs and compute abs difference * cost,
    the first position is assumed to be zero on all rics'''
    #COST USER VALUES
    cost_small_cap = 0.0020 
    cost_big_cap = 0.0010
    #CAlC
    loc_row = weight_df.index.get_loc(date_)
    C_weights = weight_df.iloc[loc_row]
    P_weights = weight_df.iloc[loc_row-1] if loc_row>0 else {key:0 for key in C_weights}
    cost_list = [np.abs(C_weights[key]-P_weights[key])*cost_big_cap for key in C_weights]
    return np.sum(cost_list)

In [ ]:
def getcurrent_optimal_universe_weight_dict(strat_, P_universe_w, C_universe_w, start_type, λ):
    '''Adjust weights of universe based on fixed value to optimize trading costs
    λ determines how much is changed each time (keeping 1-λ fixed)''' 
    #Define Starting Point  (3 scenarios, EW based on sentiment, EW random x companies)
    if P_universe_w=={}:
        if start_type[0]=="current":   # use current allocation as dictated by sentiment
            return C_universe_w
        elif start_type[0]=="random":   # use EW across random number of elements in universe
            ric_universe = list(C_universe_w)
            size_pos = len(C_universe_w) if start_type[1]<0 else start_type[1]
            selected_rics = list(set(np.random.choice(ric_universe, size=size_pos, replace=start_type[2])))
            size_pos = len(selected_rics) #adjustment as same ric can be sample multiple times
            if strat_=="L":
                return {ric: 1/size_pos for ric in C_universe_w if ric in selected_rics}
            if strat_=="S":
                return {ric: -1/size_pos for ric in C_universe_w if ric in selected_rics}            
            if strat_=="L":
                long_list = set(random.sample(selected_rics, int(size_pos/2)))
                short_list = selected_rics - long_list
                return Counter({ric: 1/size_pos for ric in C_universe_w if ric in long_list}) + Counter({ric: -1/size_pos for ric in C_universe_w if ric in short_list})
    else:
        Universe_wOptimal = {ric: C_universe_w[ric]*(λ)+P_universe_w[ric]*(1-λ) for ric in C_universe_w}
        return Universe_wOptimal

def getFULL_optimal_universe_weight_dict(Return_by_Strat_date,n_companies,   strat_,start_type=["current",0,False], λ_=1):
    optimal_weight_list_ofDict, iter_n = [], 0
    for index, row in Return_by_Strat_date.iterrows():
        Pre_OP_dict = optimal_weight_list_ofDict[iter_n-1] if (iter_n-1)>=0 else {}
        New_op_dict = getcurrent_optimal_universe_weight_dict(strat_, Pre_OP_dict, row[strat_+"("+str(n_companies)+")Weight_Universe[λ="+str(λ_)+"]"], start_type_, λ_)
        optimal_weight_list_ofDict.append(New_op_dict)
        iter_n +=1
    return optimal_weight_list_ofDict

def get_optimal_universe_weight_dict(strat_, date_, WUniverse_df, start_type=["current",0,False], λ=1):
    '''Adjust weights of universe based on fixed value to optimize trading costs
    λ determines how much is changed each time (keeping 1-λ fixed)'''
    # Get Main Data
    loc_row = WUniverse_df.index.get_loc(date_)
    C_universe_w = WUniverse_df.iloc[loc_row]
    P_universe_w = WUniverse_df.iloc[loc_row-1] 
    #Define Starting Point  (3 scenarios, EW based on sentiment, EW random x companies)
    if loc_row==0:
        if start_type[0]=="current":   # use current allocation as dictated by sentiment
            return C_universe_w
        elif start_type[0]=="random":   # use EW across random number of elements in universe
            ric_universe = list(C_universe_w)
            size_pos = len(C_universe_w) if start_type[1]<0 else start_type[1]
            selected_rics = list(set(np.random.choice(ric_universe, size=size_pos, replace=start_type[2])))
            size_pos = len(selected_rics) #adjustment as same ric can be sample multiple times
            if strat_=="L":
                return {ric: 1/size_pos for ric in C_universe_w if ric in selected_rics}
            if strat_=="S":
                return {ric: -1/size_pos for ric in C_universe_w if ric in selected_rics}            
            if strat_=="L":
                long_list = set(random.sample(selected_rics, int(size_pos/2)))
                short_list = selected_rics - long_list
                return Counter({ric: 1/size_pos for ric in C_universe_w if ric in long_list}) + Counter({ric: -1/size_pos for ric in C_universe_w if ric in short_list})
    else:
        Universe_wOptimal = Counter({ric:p_w*(1-λ) for ric, p_w in P_universe_w.items()}) + Counter({ric:c_w*(λ) for ric, c_w in C_universe_w.items()})
        return Universe_wOptimal

## Data Complements for Statistics

Fama/French 3 Factor >  https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

### Full Data Joined & without consistent beggining and end

In [ ]:
SentProxyChosen = "AbsRobust"    # Original   RelRobust   AbsRobust

In [ ]:
#  Original >    "4)Eikon_Classification/Daily_Complete_HeadlineReturn"
#  Robustness Relative Measure 1 >   "4)Eikon_Classification/Daily_Complete_HeadlineReturn_RelativeRobust"
#  Robustness Absolute Measure 2 >   "4)Eikon_Classification/Daily_Complete_HeadlineReturn_AbsoluteRobust"

#Load Saved Dataframe with IntraDay Sentiment classifications
if SentProxyChosen == "Original":
    with open("4)Eikon_Classification/Daily_Complete_HeadlineReturn", "rb") as fp:  
        InvestData = pickle.load(fp)
    print("OG")
elif SentProxyChosen == "RelRobust":
    with open("4)Eikon_Classification/Daily_Complete_HeadlineReturn_RelativeRobust", "rb") as fp:  
        InvestData = pickle.load(fp)
    print("R1")
elif SentProxyChosen == "AbsRobust":
    with open("4)Eikon_Classification/Daily_Complete_HeadlineReturn_AbsoluteRobust", "rb") as fp:  
        InvestData = pickle.load(fp)
    print("R2")

In [ ]:
# Fama French Data 5 FACTOR      Note: Data is in percentage
FF5_Data = pd.read_csv("5)InvestmentStrat/F-F_Research_Data_5_Factors_daily.csv", header=2)
FF5_Data.rename(columns={"Unnamed: 0":"Date"}, inplace=True)

# Fama French Data MOMENTUM     Note: Data is in percentage
MOM_Data = pd.read_csv("5)InvestmentStrat/F-F_Momentum_Factor_daily.csv", header=11)
MOM_Data.rename(columns={"Unnamed: 0":"Date",'Mom   ':'Mom'}, inplace=True)
MOM_Data.drop(labels=[25171],axis=0, inplace=True)
MOM_Data["Date"] = pd.to_numeric(MOM_Data["Date"])

#Merge FF Data
FamaFrench_Data = pd.merge(FF5_Data,MOM_Data, how="inner", on="Date")
print("Size 5Factor", len(FF5_Data), "  | Size MomFactor", len(MOM_Data), " | JOINED", len(FamaFrench_Data))
display(FamaFrench_Data.head(2))

#Clean Data
FamaFrench_Data["DateAdj"] = FamaFrench_Data.apply(lambda x: pd.to_datetime(str(int(x["Date"])), format='%Y-%m-%d'), axis=1)
    #(convert to decimals)
FamaFrench_Data[[col for col in FamaFrench_Data.columns if col not in ["Date","DateAdj"]]] = FamaFrench_Data[[col for col in FamaFrench_Data.columns if col not in ["Date","DateAdj"]]]/100
display(FamaFrench_Data.head(2))

In [ ]:
#Join all Data (returns & sentiment & 3FF)
Invest_FullData = pd.merge(InvestData,FamaFrench_Data, how="left", on=['DateAdj'])
display(Invest_FullData.head(2))
#Duplicate check
Invest_FullData[Invest_FullData.duplicated(subset=['DateAdj','RIC'])]

In [ ]:
save_output = input("Save Investment Data: ")
if save_output == "yes":
    if SentProxyChosen == "Original":
        with open("5)InvestmentStrat/InvestmentData_FULL", "wb") as fp:   
            pickle.dump(Invest_FullData, fp)
        print("OG")
    elif SentProxyChosen == "RelRobust":
        with open("5)InvestmentStrat/InvestmentData_FULL_RelRobust", "wb") as fp:   
            pickle.dump(Invest_FullData, fp)
        print("R1")
    elif SentProxyChosen == "AbsRobust":
        with open("5)InvestmentStrat/InvestmentData_FULL_AbsRobust", "wb") as fp:   
            pickle.dump(Invest_FullData, fp)
        print("R2")

### Aligned Pivot Tables for Portfolios Creation

In [ ]:
SentProxyChosen = "AbsRobust"    # Original   RelRobust   AbsRobust

In [ ]:
#Load Saved Dataframe with IntraDay Sentiment classifications
if SentProxyChosen == "Original":
    with open("5)InvestmentStrat/InvestmentData_FULL", "rb") as fp:  
        Invest_FullData = pickle.load(fp)
    print("OG")
elif SentProxyChosen == "RelRobust":
    with open("5)InvestmentStrat/InvestmentData_FULL_RelRobust", "rb") as fp:  
        Invest_FullData = pickle.load(fp)
    print("R1")
elif SentProxyChosen == "AbsRobust":
    with open("5)InvestmentStrat/InvestmentData_FULL_AbsRobust", "rb") as fp:  
        Invest_FullData = pickle.load(fp)
    print("R2")

In [ ]:
#For Each Eliminate rows if All NULL
    #Return by date and company
Ret_date_company = Invest_FullData.pivot(index='DateAdj', columns='RIC', values='RET')  
print("Ret PIVOT", len(Ret_date_company), " Dropped Dates", list(Ret_date_company[Ret_date_company.isnull().all(axis=1)].index))
Ret_date_company = Ret_date_company[~Ret_date_company.isnull().all(axis=1)].copy()
    #Lagged sent by date and company
LagSent_date_company = Invest_FullData.pivot(index='DateAdj', columns='RIC', values='DailyAvgSent(t-1)') 
print("L Sent PIVOT", len(LagSent_date_company), " Dropped Dates", list(LagSent_date_company[LagSent_date_company.isnull().all(axis=1)].index))
LagSent_date_company = LagSent_date_company[~LagSent_date_company.isnull().all(axis=1)].copy()
    #Lagged Market Cap by date and company
LagMktCap_date_company = Invest_FullData.pivot(index='DateAdj', columns='RIC', values='MktCap(t-1)') 
print("L MktCap PIVOT", len(LagMktCap_date_company), " Dropped Dates", list(LagMktCap_date_company[LagMktCap_date_company.isnull().all(axis=1)].index))
LagMktCap_date_company = LagMktCap_date_company[~LagMktCap_date_company.isnull().all(axis=1)].copy()

#Keep only dates in common
common_dates = sorted(list(set(set(Ret_date_company.index).intersection(LagSent_date_company.index)).intersection(LagMktCap_date_company.index)))
Ret_date_company = Ret_date_company.loc[common_dates]
LagSent_date_company = LagSent_date_company.loc[common_dates]
LagMktCap_date_company = LagMktCap_date_company.loc[common_dates]
Invest_FullData_Limited = Invest_FullData[Invest_FullData["DateAdj"].isin(common_dates)].drop_duplicates(subset=["DateAdj"])
print("Final Size", len(Ret_date_company), len(LagSent_date_company), len(LagMktCap_date_company))
print("Slimmed Stats", len(Invest_FullData_Limited))

#Save all in Dict Variable
Invest_Pivots = {"Ret":Ret_date_company,
                 "LagSent":LagSent_date_company,
                 "LagMktCap":LagMktCap_date_company,
                 "InvestData":Invest_FullData_Limited}

In [ ]:
save_output = input("Save Investment Data: ")
if save_output == "yes":
    if SentProxyChosen == "Original":
        with open("5)InvestmentStrat/InvestmentData_PivotsDict", "wb") as fp:   
            pickle.dump(Invest_Pivots, fp)
        print("OG")
    elif SentProxyChosen == "RelRobust":
        with open("5)InvestmentStrat/InvestmentData_PivotsDict_RelRobust", "wb") as fp:   
            pickle.dump(Invest_Pivots, fp)
        print("R1")
    elif SentProxyChosen == "AbsRobust":
        with open("5)InvestmentStrat/InvestmentData_PivotsDict_AbsRobust", "wb") as fp:   
            pickle.dump(Invest_Pivots, fp)
        print("R2")

## Initiate NOCost Data Save

In [ ]:
SentProxyChosenInv = "AbsRobust"    # Original   RelRobust   AbsRobust

In [ ]:
#SAVE DATA
save_data_file_nocost = input("Create File to Save the Data?")
if save_data_file_nocost =="yes":
    datestring_time = datetime.strftime(datetime.now(),"%m_%d_%Y")
    if SentProxyChosenInv=="Original":
        NoCost_writer = pd.ExcelWriter("3)Tables_PLots/" + str(datestring_time) + "__Investment_Strat_NoCOST.xlsx", engine='xlsxwriter')
        file_name = "InvestmentData_PivotsDict"
    elif SentProxyChosenInv=="RelRobust":
        NoCost_writer = pd.ExcelWriter("3)Tables_PLots/" + str(datestring_time) + "__Investment_Strat_NoCOST_RelRobust.xlsx", engine='xlsxwriter')
        file_name = "InvestmentData_PivotsDict_RelRobust"
    elif SentProxyChosenInv=="AbsRobust":
        NoCost_writer = pd.ExcelWriter("3)Tables_PLots/" + str(datestring_time) + "__Investment_Strat_NoCOST_AbsRobust.xlsx", engine='xlsxwriter')
        file_name = "InvestmentData_PivotsDict_AbsRobust"
        
save_data_nocost = input("Save the Data?")
if save_data_nocost =="yes":
    NoCost_writer.save()
    
save_data_nocost = "yes"

## Benchmark
Buy and Hold S&P

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]

In [ ]:
#Get Full data of returns for the period for which the sent startegy is applicable
Buy_Hold = Invest_FullData.drop_duplicates(subset=["DateAdj"])
Buy_Hold = Buy_Hold.sort_values(by='DateAdj')
#Cumulative
Buy_Hold["Cumulative"] = (Buy_Hold["sprtrn"]+1).cumprod()
#Visual
plt.figure(figsize=(20, 5))
plt.plot(Buy_Hold["DateAdj"],Buy_Hold["Cumulative"])
#Table snippet
display(Buy_Hold[['DateAdj','sprtrn', 'Cumulative', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom']].head(2))

In [ ]:
#Data
df = paper_stats_table("Buy & Hold", "sprtrn","Cumulative",  Buy_Hold)
display(df)
strat_data = pd.DataFrame()
col_list = list(df.columns)
for col in col_list:
    for index, row in df.iterrows():
        value = df.loc[index,col]
        if not(pd.isnull(value)):
            strat_data.loc["Buy-Hold",f"{col}{index}"] = value
#Save
if save_data_nocost.lower() == "yes":
    strat_data.to_excel(NoCost_writer, sheet_name='S&P',startrow=1)

## Basic Main Strats

**No Transaction Costs**

### Equal Weighted
L, S, L-S   <>  Max 100 stocks

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]

In [ ]:
#Construct List of top and bottom companies for portfolio creation
#(auxiliary)
n_companies = 100
unique_rics = list(LagSent_date_company.columns)
Return_by_Strat_date = pd.DataFrame()  
#Iterate over each type of strat and compute statistics
for strat_ in ["L", "S", "L-S"]:
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict"] = LagSent_date_company.apply(lambda x: get_ric_pos_dict(strat_,n_companies,x.name,x,[Ret_date_company])  , axis=1)  
    Return_by_Strat_date.sort_index(inplace=True)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")NCompanies"] = Return_by_Strat_date.apply(lambda x:  {"L":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == 1),"S":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == -1)} , axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RET_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], Ret_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"] = Return_by_Strat_date.apply(lambda x: get_universe_weight_dict(unique_rics, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], None),  axis=1)    
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_List"] = Return_by_Strat_date.apply(lambda x: [x[strat_+"("+str(n_companies)+")Weight_Universe"][key] for key in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"]], axis=1)
    Return_by_Strat_date[strat_+" Turnover"] = Return_by_Strat_date.apply(lambda x: get_daily_turnover(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"],Ret_date_company), axis=1)
    Return_by_Strat_date[strat_] = Return_by_Strat_date.apply(lambda x: sum(ret * w for ret, w in zip(x[strat_+"("+str(n_companies)+")RET_List"], x[strat_+"("+str(n_companies)+")Weight_List"])),  axis=1)
    Return_by_Strat_date[strat_+" Cumulative"] =  (Return_by_Strat_date[strat_]+1).cumprod()

In [ ]:
#Data with info for Stats
Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),Invest_FullData[["DateAdj",'MktCap(t-1)','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']], how="inner", on="DateAdj")
print("Data Final", len(Portfolio_Stats_Data), "Data Port", len(Return_by_Strat_date), "Data Baseline", len(Invest_FullData))
display(Portfolio_Stats_Data.head(2))

In [ ]:
#Strat Stats
strat_data = pd.DataFrame()
plt.figure(figsize=(20, 5))
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index_s, strat in enumerate(['L', 'S', 'L-S']):  
    #Data
    df = paper_stats_table(strat, strat, strat+" Cumulative",  Portfolio_Stats_Data)
    df.loc[strat, "Turnover (%)"] = np.mean(Portfolio_Stats_Data[strat+" Turnover"])*100
    display(df)
    col_list = list(df.columns)
    for col in col_list:
        for index, row in df.iterrows():
            value = df.loc[index,col]
            if not(pd.isnull(value)):
                index_name = "" if index in  ['L', 'S', 'L-S'] else index
                strat_data.loc[strat,f"{col}{index_name}"] = value
    plt.plot(Portfolio_Stats_Data["DateAdj"],Portfolio_Stats_Data[strat+" Cumulative"], label = strat, color='tab:'+color[index_s])
plt.xlabel('Date')
plt.title("EW Base Startegies")
plt.legend()
plt.show()

#Save
if save_data_nocost.lower() == "yes":
    strat_data.to_excel(NoCost_writer, sheet_name='EW',startrow=1)

### Value Weighted
L, S, L-S   <>  Max 100 stocks

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]

In [ ]:
#Construct List of top and bottom companies for portfolio creation
#(auxiliary)
n_companies = 100
unique_rics = list(LagSent_date_company.columns)
Return_by_Strat_date = pd.DataFrame()  
#Iterate over each type of strat and compute statistics
for strat_ in ["L", "S", "L-S"]:
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict"] = LagSent_date_company.apply(lambda x: get_ric_pos_dict(strat_,n_companies,x.name,x,[Ret_date_company, LagMktCap_date_company])  , axis=1)  
    Return_by_Strat_date.sort_index(inplace=True)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")NCompanies"] = Return_by_Strat_date.apply(lambda x:  {"L":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == 1),"S":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == -1)} , axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RET_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], Ret_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")MKT_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], LagMktCap_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"] = Return_by_Strat_date.apply(lambda x: get_universe_weight_dict(unique_rics, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], x[strat_+"("+str(n_companies)+")MKT_List"]),  axis=1)    
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_List"] = Return_by_Strat_date.apply(lambda x: [x[strat_+"("+str(n_companies)+")Weight_Universe"][key] for key in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"]], axis=1)
    Return_by_Strat_date[strat_+" Turnover"] = Return_by_Strat_date.apply(lambda x: get_daily_turnover(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"],Ret_date_company), axis=1)
    Return_by_Strat_date[strat_] = Return_by_Strat_date.apply(lambda x: sum(ret * w for ret, w in zip(x[strat_+"("+str(n_companies)+")RET_List"], x[strat_+"("+str(n_companies)+")Weight_List"])),  axis=1)
    Return_by_Strat_date[strat_+" Cumulative"] =  (Return_by_Strat_date[strat_]+1).cumprod()

In [ ]:
#Data with info for Stats
Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),Invest_FullData[["DateAdj",'MktCap(t-1)','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']], how="inner", on="DateAdj")
print("Data Final", len(Portfolio_Stats_Data), "Data Port", len(Return_by_Strat_date), "Data Baseline", len(Invest_FullData))
display(Portfolio_Stats_Data.head(2))

In [ ]:
#Strat Stats
strat_data = pd.DataFrame()
plt.figure(figsize=(20, 5))
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index_s, strat in enumerate(['L', 'S', 'L-S']):  
    #Data
    df = paper_stats_table(strat, strat, strat+" Cumulative",  Portfolio_Stats_Data)
    df.loc[strat, "Turnover (%)"] = np.mean(Portfolio_Stats_Data[strat+" Turnover"])*100
    display(df)
    col_list = list(df.columns)
    for col in col_list:
        for index, row in df.iterrows():
            value = df.loc[index,col]
            if not(pd.isnull(value)):
                index_name = "" if index in  ['L', 'S', 'L-S'] else index
                strat_data.loc[strat,f"{col}{index_name}"] = value
    plt.plot(Portfolio_Stats_Data["DateAdj"],Portfolio_Stats_Data[strat+" Cumulative"], label = strat, color='tab:'+color[index_s])
plt.xlabel('Date')
plt.title("VW Base Startegies")
plt.legend()
plt.show()

#Save
if save_data_nocost.lower() == "yes":
    strat_data.to_excel(NoCost_writer, sheet_name='VW',startrow=1)

## Return Response on Shifted Sentiment

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]   #return at t
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]
#Additional needed
Sent_date_company = LagSent_date_company.shift(periods=-1)  #sentiment at t
PreviousRet_date_company = Ret_date_company.shift(periods=1) #ret at t-1
NextRet_date_company = Ret_date_company.shift(periods=-1) #ret at t+1
# FF
FF_Data = Invest_FullData[["DateAdj",'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']].set_index("DateAdj")
PreviousFF_Data = FF_Data.shift(periods=1)
NextFF_Data = FF_Data.shift(periods=-1)

In [ ]:
#Construct List of top and bottom companies for portfolio creation
#(auxiliary)
n_companies = 100
unique_rics = list(Sent_date_company.columns)
Return_by_Strat_date = pd.DataFrame()  
# Loop over different Lagges
for RET_Data, id_ in zip([PreviousRet_date_company,Ret_date_company,NextRet_date_company],["_D-1","_D0","_D+1"]):
    print("Computing stats for", id_)
    #Iterate over each type of strat and compute statistics
    for strat_ in ["L", "S", "L-S"]:
        Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict"+id_] = Sent_date_company.apply(lambda x: get_ric_pos_dict(strat_,n_companies,x.name,x,[RET_Data])  , axis=1).replace('None',np.NaN).fillna(method="ffill")  
        Return_by_Strat_date.sort_index(inplace=True)
        Return_by_Strat_date[strat_+"("+str(n_companies)+")NCompanies"+id_] = Return_by_Strat_date.apply(lambda x:  {"L":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"+id_].values() if v == 1),"S":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"+id_].values() if v == -1)} , axis=1)
        Return_by_Strat_date[strat_+"("+str(n_companies)+")RET_List"+id_] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"+id_], RET_Data), axis=1)
        Return_by_Strat_date[strat_+"("+str(n_companies)+")MKT_List"+id_] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"+id_], LagMktCap_date_company), axis=1)
        Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"+id_] = Return_by_Strat_date.apply(lambda x: get_universe_weight_dict(unique_rics, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"+id_], None),  axis=1)     
        Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_List"+id_] = Return_by_Strat_date.apply(lambda x: [x[strat_+"("+str(n_companies)+")Weight_Universe"+id_][key] for key in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"+id_]], axis=1)
        Return_by_Strat_date[strat_+id_+" Turnover"] = Return_by_Strat_date.apply(lambda x: get_daily_turnover(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"+id_],RET_Data), axis=1)
        Return_by_Strat_date[strat_+id_] = Return_by_Strat_date.apply(lambda x: sum(ret * w for ret, w in zip(x[strat_+"("+str(n_companies)+")RET_List"+id_], x[strat_+"("+str(n_companies)+")Weight_List"+id_])),  axis=1)
        Return_by_Strat_date[strat_+" Cumulative"+id_] =  (Return_by_Strat_date[strat_+id_]+1).cumprod()

In [ ]:
#Data with info for Stats
Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),Invest_FullData[["DateAdj",'MktCap(t-1)','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']], how="inner", on="DateAdj")
print("Data Final", len(Portfolio_Stats_Data), "Data Port", len(Return_by_Strat_date), "Data Baseline", len(Invest_FullData))
display(Portfolio_Stats_Data.head(2))

In [ ]:
#Strat Stats
plt.figure(figsize=(20, 5))
color = ["grey","red","blue"]
strat_type = ['L', 'S', 'L-S']
line_type = ['-.','-','dotted']
shift = ["_D-1","_D0","_D+1"]
for strat in ['L', 'S', 'L-S']:
    for t in ["_D-1","_D0","_D+1"]:
        strat_var = strat+" Cumulative"+t
        plt.plot(Portfolio_Stats_Data["DateAdj"],Portfolio_Stats_Data[strat_var], label = strat_var, linestyle=line_type[shift.index(t)], color='tab:'+color[strat_type.index(strat)])
plt.xlabel('Date')
#plt.title("Shifted Sent Impact")
plt.legend()
if save_data_nocost.lower() == "yes":
    plt.savefig('3)Tables_PLots/SpeedInfoAssimilation_'+SentProxyChosenInv+'.png')
plt.show()

In [ ]:
strat_data = pd.DataFrame()
for t, famafrench_df in zip(["_D-1","_D0","_D+1"],[PreviousFF_Data, FF_Data, NextFF_Data]):
    famafrench_df.dropna(how='all', inplace=True)
    Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),famafrench_df.reset_index(), how="inner", on="DateAdj")
    for strat in ['L', 'S', 'L-S']:
        df = paper_stats_table(strat+t, strat+t, strat+" Cumulative"+t,  Portfolio_Stats_Data)
        df.loc[strat+t, "Turnover (%)"] = np.mean(Portfolio_Stats_Data[strat+t+" Turnover"])*100
        display(df)
        col_list = list(df.columns)
        for col in col_list:
            for index, row in df.iterrows():
                value = df.loc[index,col]
                if not(pd.isnull(value)):
                    index_name = "" if index == strat+t else index
                    strat_data.loc[strat+t,f"{col}{index_name}"] = value
        
        
#Save stats
if save_data_nocost.lower() == "yes": 
    # Position the dataframes in the worksheet.
    strat_data.to_excel(NoCost_writer, sheet_name='InfoAssimilation'+strat,startrow=1)

## Alternative Strats

### 200 stocks (EW)

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]

In [ ]:
#Construct List of top and bottom companies for portfolio creation
#(auxiliary)
n_companies = 200
unique_rics = list(LagSent_date_company.columns)
Return_by_Strat_date = pd.DataFrame()  
#Iterate over each type of strat and compute statistics
for strat_ in ["L", "S", "L-S"]:
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict"] = LagSent_date_company.apply(lambda x: get_ric_pos_dict(strat_,n_companies,x.name,x,[Ret_date_company])  , axis=1)  
    Return_by_Strat_date.sort_index(inplace=True)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")NCompanies"] = Return_by_Strat_date.apply(lambda x:  {"L":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == 1),"S":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == -1)} , axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RET_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], Ret_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"] = Return_by_Strat_date.apply(lambda x: get_universe_weight_dict(unique_rics, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], None),  axis=1)    
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_List"] = Return_by_Strat_date.apply(lambda x: [x[strat_+"("+str(n_companies)+")Weight_Universe"][key] for key in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"]], axis=1)
    Return_by_Strat_date[strat_+" Turnover"] = Return_by_Strat_date.apply(lambda x: get_daily_turnover(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"],Ret_date_company), axis=1)
    Return_by_Strat_date[strat_] = Return_by_Strat_date.apply(lambda x: sum(ret * w for ret, w in zip(x[strat_+"("+str(n_companies)+")RET_List"], x[strat_+"("+str(n_companies)+")Weight_List"])),  axis=1)
    Return_by_Strat_date[strat_+" Cumulative"] =  (Return_by_Strat_date[strat_]+1).cumprod()

In [ ]:
#Data with info for Stats
Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),Invest_FullData[["DateAdj",'MktCap(t-1)','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']], how="inner", on="DateAdj")
print("Data Final", len(Portfolio_Stats_Data), "Data Port", len(Return_by_Strat_date), "Data Baseline", len(Invest_FullData))
display(Portfolio_Stats_Data.head(2))

In [ ]:
#Strat Stats
strat_data = pd.DataFrame()
plt.figure(figsize=(20, 5))
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index_s, strat in enumerate(['L', 'S', 'L-S']):  
    #Data
    df = paper_stats_table(strat, strat, strat+" Cumulative",  Portfolio_Stats_Data)
    df.loc[strat, "Turnover (%)"] = np.mean(Portfolio_Stats_Data[strat+" Turnover"])*100
    display(df)
    col_list = list(df.columns)
    for col in col_list:
        for index, row in df.iterrows():
            value = df.loc[index,col]
            if not(pd.isnull(value)):
                index_name = "" if index in  ['L', 'S', 'L-S'] else index
                strat_data.loc[strat,f"{col}{index_name}"] = value
    plt.plot(Portfolio_Stats_Data["DateAdj"],Portfolio_Stats_Data[strat+" Cumulative"], label = strat, color='tab:'+color[index_s])
plt.xlabel('Date')
plt.title("200 stocks Base Startegies")
plt.legend()
plt.show()

#Save
if save_data_nocost.lower() == "yes":
    strat_data.to_excel(NoCost_writer, sheet_name='Stocks200EW',startrow=1)

### 200 stocks (VW)

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]

In [ ]:
#Construct List of top and bottom companies for portfolio creation
#(auxiliary)
n_companies = 200
unique_rics = list(LagSent_date_company.columns)
Return_by_Strat_date = pd.DataFrame()  
#Iterate over each type of strat and compute statistics
for strat_ in ["L", "S", "L-S"]:
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict"] = LagSent_date_company.apply(lambda x: get_ric_pos_dict(strat_,n_companies,x.name,x,[Ret_date_company, LagMktCap_date_company])  , axis=1)  
    Return_by_Strat_date.sort_index(inplace=True)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")NCompanies"] = Return_by_Strat_date.apply(lambda x:  {"L":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == 1),"S":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == -1)} , axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RET_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], Ret_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")MKT_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], LagMktCap_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"] = Return_by_Strat_date.apply(lambda x: get_universe_weight_dict(unique_rics, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], x[strat_+"("+str(n_companies)+")MKT_List"]),  axis=1)    
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_List"] = Return_by_Strat_date.apply(lambda x: [x[strat_+"("+str(n_companies)+")Weight_Universe"][key] for key in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"]], axis=1)
    Return_by_Strat_date[strat_+" Turnover"] = Return_by_Strat_date.apply(lambda x: get_daily_turnover(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"],Ret_date_company), axis=1)
    Return_by_Strat_date[strat_] = Return_by_Strat_date.apply(lambda x: sum(ret * w for ret, w in zip(x[strat_+"("+str(n_companies)+")RET_List"], x[strat_+"("+str(n_companies)+")Weight_List"])),  axis=1)
    Return_by_Strat_date[strat_+" Cumulative"] =  (Return_by_Strat_date[strat_]+1).cumprod()

In [ ]:
#Data with info for Stats
Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),Invest_FullData[["DateAdj",'MktCap(t-1)','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']], how="inner", on="DateAdj")
print("Data Final", len(Portfolio_Stats_Data), "Data Port", len(Return_by_Strat_date), "Data Baseline", len(Invest_FullData))
display(Portfolio_Stats_Data.head(2))

In [ ]:
#Strat Stats
strat_data = pd.DataFrame()
plt.figure(figsize=(20, 5))
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index_s, strat in enumerate(['L', 'S', 'L-S']):  
    #Data
    df = paper_stats_table(strat, strat, strat+" Cumulative",  Portfolio_Stats_Data)
    df.loc[strat, "Turnover (%)"] = np.mean(Portfolio_Stats_Data[strat+" Turnover"])*100
    display(df)
    col_list = list(df.columns)
    for col in col_list:
        for index, row in df.iterrows():
            value = df.loc[index,col]
            if not(pd.isnull(value)):
                index_name = "" if index in  ['L', 'S', 'L-S'] else index
                strat_data.loc[strat,f"{col}{index_name}"] = value
    plt.plot(Portfolio_Stats_Data["DateAdj"],Portfolio_Stats_Data[strat+" Cumulative"], label = strat, color='tab:'+color[index_s])
plt.xlabel('Date')
plt.title("200 stocks Base Startegies")
plt.legend()
plt.show()

#Save
if save_data_nocost.lower() == "yes":
    strat_data.to_excel(NoCost_writer, sheet_name='Stocks200VW',startrow=1)

### Follow the Loser (EW)

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]

In [ ]:
#Construct List of top and bottom companies for portfolio creation
#(auxiliary)
n_companies = 100
unique_rics = list(LagSent_date_company.columns)
Return_by_Strat_date = pd.DataFrame()  
#Iterate over each type of strat and compute statistics
for strat_ in ["L", "S", "L-S"]:
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict_FW"] = LagSent_date_company.apply(lambda x: get_ric_pos_dict(strat_,n_companies,x.name,x,[Ret_date_company])  , axis=1)  
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict"] = Return_by_Strat_date.apply(lambda x: {key:value*(-1) for key, value in x [strat_+"("+str(n_companies)+")RIC_Pos_Dict_FW"].items()}, axis=1) 
    Return_by_Strat_date.sort_index(inplace=True)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")NCompanies"] = Return_by_Strat_date.apply(lambda x:  {"L":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == 1),"S":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == -1)} , axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RET_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], Ret_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"] = Return_by_Strat_date.apply(lambda x: get_universe_weight_dict(unique_rics, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], None),  axis=1)    
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_List"] = Return_by_Strat_date.apply(lambda x: [x[strat_+"("+str(n_companies)+")Weight_Universe"][key] for key in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"]], axis=1)
    Return_by_Strat_date[strat_+" Turnover"] = Return_by_Strat_date.apply(lambda x: get_daily_turnover(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"],Ret_date_company), axis=1)
    Return_by_Strat_date[strat_] = Return_by_Strat_date.apply(lambda x: sum(ret * w for ret, w in zip(x[strat_+"("+str(n_companies)+")RET_List"], x[strat_+"("+str(n_companies)+")Weight_List"])),  axis=1)
    Return_by_Strat_date[strat_+" Cumulative"] =  (Return_by_Strat_date[strat_]+1).cumprod()

In [ ]:
#Data with info for Stats
Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),Invest_FullData[["DateAdj",'MktCap(t-1)','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']], how="inner", on="DateAdj")
print("Data Final", len(Portfolio_Stats_Data), "Data Port", len(Return_by_Strat_date), "Data Baseline", len(Invest_FullData))
display(Portfolio_Stats_Data.head(2))

In [ ]:
#Strat Stats
strat_data = pd.DataFrame()
plt.figure(figsize=(20, 5))
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index_s, strat in enumerate(['L', 'S', 'L-S']):  
    #Data
    df = paper_stats_table(strat, strat, strat+" Cumulative",  Portfolio_Stats_Data)
    df.loc[strat, "Turnover (%)"] = np.mean(Portfolio_Stats_Data[strat+" Turnover"])*100
    display(df)
    col_list = list(df.columns)
    for col in col_list:
        for index, row in df.iterrows():
            value = df.loc[index,col]
            if not(pd.isnull(value)):
                index_name = "" if index in  ['L', 'S', 'L-S'] else index
                strat_data.loc[strat,f"{col}{index_name}"] = value
    plt.plot(Portfolio_Stats_Data["DateAdj"],Portfolio_Stats_Data[strat+" Cumulative"], label = strat, color='tab:'+color[index_s])
plt.xlabel('Date')
plt.title("200 stocks Base Startegies")
plt.legend()
plt.show()


#Save
if save_data_nocost.lower() == "yes":
    strat_data.to_excel(NoCost_writer, sheet_name='FollowLoserEW',startrow=1)

### Follow the Loser (VW)

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]

In [ ]:
#Construct List of top and bottom companies for portfolio creation
#(auxiliary)
n_companies = 100
unique_rics = list(LagSent_date_company.columns)
Return_by_Strat_date = pd.DataFrame()  
#Iterate over each type of strat and compute statistics
for strat_ in ["L", "S", "L-S"]:
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict_FW"] = LagSent_date_company.apply(lambda x: get_ric_pos_dict(strat_,n_companies,x.name,x,[Ret_date_company, LagMktCap_date_company])  , axis=1)  
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict"] = Return_by_Strat_date.apply(lambda x: {key:value*(-1) for key, value in x [strat_+"("+str(n_companies)+")RIC_Pos_Dict_FW"].items()}, axis=1) 
    Return_by_Strat_date.sort_index(inplace=True)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")NCompanies"] = Return_by_Strat_date.apply(lambda x:  {"L":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == 1),"S":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == -1)} , axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RET_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], Ret_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")MKT_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], LagMktCap_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"] = Return_by_Strat_date.apply(lambda x: get_universe_weight_dict(unique_rics, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], x[strat_+"("+str(n_companies)+")MKT_List"]),  axis=1)    
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_List"] = Return_by_Strat_date.apply(lambda x: [x[strat_+"("+str(n_companies)+")Weight_Universe"][key] for key in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"]], axis=1)
    Return_by_Strat_date[strat_+" Turnover"] = Return_by_Strat_date.apply(lambda x: get_daily_turnover(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"],Ret_date_company), axis=1)
    Return_by_Strat_date[strat_] = Return_by_Strat_date.apply(lambda x: sum(ret * w for ret, w in zip(x[strat_+"("+str(n_companies)+")RET_List"], x[strat_+"("+str(n_companies)+")Weight_List"])),  axis=1)
    Return_by_Strat_date[strat_+" Cumulative"] =  (Return_by_Strat_date[strat_]+1).cumprod()

In [ ]:
#Data with info for Stats
Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),Invest_FullData[["DateAdj",'MktCap(t-1)','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']], how="inner", on="DateAdj")
print("Data Final", len(Portfolio_Stats_Data), "Data Port", len(Return_by_Strat_date), "Data Baseline", len(Invest_FullData))
display(Portfolio_Stats_Data.head(2))

In [ ]:
#Strat Stats
strat_data = pd.DataFrame()
plt.figure(figsize=(20, 5))
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index_s, strat in enumerate(['L', 'S', 'L-S']):  
    #Data
    df = paper_stats_table(strat, strat, strat+" Cumulative",  Portfolio_Stats_Data)
    df.loc[strat, "Turnover (%)"] = np.mean(Portfolio_Stats_Data[strat+" Turnover"])*100
    display(df)
    col_list = list(df.columns)
    for col in col_list:
        for index, row in df.iterrows():
            value = df.loc[index,col]
            if not(pd.isnull(value)):
                index_name = "" if index in  ['L', 'S', 'L-S'] else index
                strat_data.loc[strat,f"{col}{index_name}"] = value
    plt.plot(Portfolio_Stats_Data["DateAdj"],Portfolio_Stats_Data[strat+" Cumulative"], label = strat, color='tab:'+color[index_s])
plt.xlabel('Date')
plt.title("200 stocks Base Startegies")
plt.legend()
plt.show()

#Save
if save_data_nocost.lower() == "yes":
    strat_data.to_excel(NoCost_writer, sheet_name='FollowLoserVW',startrow=1)

## Initiate NoCost Data Save

In [ ]:
SentProxyChosenInvCOST = "AbsRobust"    # Original   RelRobust   AbsRobust

In [ ]:
#SAVE DATA
save_data_file_cost = input("Create File to Save the Data?")
if save_data_file_cost =="yes":
    datestring_time = datetime.strftime(datetime.now(),"%m_%d_%Y")
    if SentProxyChosenInvCOST=="Original":
        Cost_writer = pd.ExcelWriter("3)Tables_PLots/" + str(datestring_time) + "__Investment_Strat_COST.xlsx", engine='xlsxwriter')
        file_name = "InvestmentData_PivotsDict"
    elif SentProxyChosenInvCOST=="RelRobust":
        Cost_writer = pd.ExcelWriter("3)Tables_PLots/" + str(datestring_time) + "__Investment_Strat_COST_RelRobust.xlsx", engine='xlsxwriter')
        file_name = "InvestmentData_PivotsDict_RelRobust"
    elif SentProxyChosenInvCOST=="AbsRobust":
        Cost_writer = pd.ExcelWriter("3)Tables_PLots/" + str(datestring_time) + "__Investment_Strat_COST_AbsRobust.xlsx", engine='xlsxwriter')
        file_name = "InvestmentData_PivotsDict_AbsRobust"
        
save_data_cost = input("Save the Data?")
if save_data_cost =="yes":
    Cost_writer.save()
    
save_data_cost = "yes"

## Economic Viability Portfolios (Basic)

**Transaction Costs**

### Applied to EW Basic Strats 

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]

In [ ]:
#Construct List of top and bottom companies for portfolio creation
#(auxiliary)
n_companies = 100
unique_rics = list(LagSent_date_company.columns)
Return_by_Strat_date = pd.DataFrame()  
#Iterate over each type of strat and compute statistics
for strat_ in ["L", "S", "L-S"]:
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict"] = LagSent_date_company.apply(lambda x: get_ric_pos_dict(strat_,n_companies,x.name,x,[Ret_date_company])  , axis=1)  
    Return_by_Strat_date.sort_index(inplace=True)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")NCompanies"] = Return_by_Strat_date.apply(lambda x:  {"L":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == 1),"S":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == -1)} , axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RET_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], Ret_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"] = Return_by_Strat_date.apply(lambda x: get_universe_weight_dict(unique_rics, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], None),  axis=1)    
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_List"] = Return_by_Strat_date.apply(lambda x: [x[strat_+"("+str(n_companies)+")Weight_Universe"][key] for key in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"]], axis=1)
    Return_by_Strat_date[strat_] = Return_by_Strat_date.apply(lambda x: sum(ret * w for ret, w in zip(x[strat_+"("+str(n_companies)+")RET_List"], x[strat_+"("+str(n_companies)+")Weight_List"])),  axis=1)
    Return_by_Strat_date[strat_+" Cumulative"] =  (Return_by_Strat_date[strat_]+1).cumprod()
    Return_by_Strat_date[strat_+" Costs"] = Return_by_Strat_date.apply(lambda x: get_transaction_costs(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"]), axis=1)
    Return_by_Strat_date[strat_+" Net"] =  Return_by_Strat_date[strat_] - Return_by_Strat_date[strat_+" Costs"]

In [ ]:
#Data with info for Stats
Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),Invest_FullData[["DateAdj",'MktCap(t-1)','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']], how="inner", on="DateAdj")
print("Data Final", len(Portfolio_Stats_Data), "Data Port", len(Return_by_Strat_date), "Data Baseline", len(Invest_FullData))
display(Portfolio_Stats_Data.head(2))

In [ ]:
#Strat Stats
strat_data = pd.DataFrame()
for strat in ['L', 'S', 'L-S']:
    print( "\t Strat: ",strat)
    df = paper_stats_table_Costs(strat, strat, strat+" Net",  Portfolio_Stats_Data)
    display(df)
    col_list = list(df.columns)
    for col in col_list:
        for index, row in df.iterrows():
            index_name = index if index not in  ['L', 'S', 'L-S'] else ""
            strat_data.loc[strat,f"{col}{index_name}"] = df.loc[index,col]
#Save
strat_data.to_excel(Cost_writer, sheet_name=str(n_companies)+'EWCost',startrow=1)

### Applied to VW Basic Strats 

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]

In [ ]:
#Construct List of top and bottom companies for portfolio creation
#(auxiliary)
n_companies = 100
unique_rics = list(LagSent_date_company.columns)
Return_by_Strat_date = pd.DataFrame()  
#Iterate over each type of strat and compute statistics
for strat_ in ["L", "S", "L-S"]:
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict"] = LagSent_date_company.apply(lambda x: get_ric_pos_dict(strat_,n_companies,x.name,x,[Ret_date_company, LagMktCap_date_company])  , axis=1)  
    Return_by_Strat_date.sort_index(inplace=True)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")NCompanies"] = Return_by_Strat_date.apply(lambda x:  {"L":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == 1),"S":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"].values() if v == -1)} , axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")RET_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], Ret_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")MKT_List"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], LagMktCap_date_company), axis=1)
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"] = Return_by_Strat_date.apply(lambda x: get_universe_weight_dict(unique_rics, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"], x[strat_+"("+str(n_companies)+")MKT_List"]),  axis=1)    
    Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_List"] = Return_by_Strat_date.apply(lambda x: [x[strat_+"("+str(n_companies)+")Weight_Universe"][key] for key in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict"]], axis=1)
    Return_by_Strat_date[strat_] = Return_by_Strat_date.apply(lambda x: sum(ret * w for ret, w in zip(x[strat_+"("+str(n_companies)+")RET_List"], x[strat_+"("+str(n_companies)+")Weight_List"])),  axis=1)
    Return_by_Strat_date[strat_+" Cumulative"] =  (Return_by_Strat_date[strat_]+1).cumprod()
    Return_by_Strat_date[strat_+" Costs"] = Return_by_Strat_date.apply(lambda x: get_transaction_costs(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe"]), axis=1)
    Return_by_Strat_date[strat_+" Net"] =  Return_by_Strat_date[strat_] - Return_by_Strat_date[strat_+" Costs"]

In [ ]:
#Data with info for Stats
Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),Invest_FullData[["DateAdj",'MktCap(t-1)','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']], how="inner", on="DateAdj")
print("Data Final", len(Portfolio_Stats_Data), "Data Port", len(Return_by_Strat_date), "Data Baseline", len(Invest_FullData))
display(Portfolio_Stats_Data.head(2))

In [ ]:
#Strat Stats
strat_data = pd.DataFrame()
for strat in ['L', 'S', 'L-S']:
    print( "\t Strat: ",strat)
    df = paper_stats_table_Costs(strat, strat, strat+" Net",  Portfolio_Stats_Data)
    display(df)
    col_list = list(df.columns)
    for col in col_list:
        for index, row in df.iterrows():
            index_name = index if index not in  ['L', 'S', 'L-S'] else ""
            strat_data.loc[strat,f"{col}{index_name}"] = df.loc[index,col]
#Save
strat_data.to_excel(Cost_writer, sheet_name=str(n_companies)+'VWCost',startrow=1)

## Economic Viability Portfolios (OPTIMMIZED)

**Transaction Costs**
Fixed percentage adjusted each time (theta)

### Start Based on 1st Signal

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]

In [ ]:
#Construct List of top and bottom companies for portfolio creation  
#(auxiliary)
n_companies = 100
start_type_ = ["current",n_companies,True]   # first element can be  "current"   or "random"
unique_rics = list(LagSent_date_company.columns)
Return_by_Strat_date = pd.DataFrame()  
# Loop over different lambdas (percentage of positions to change) max 1 (= change everthing) and min 0 (=change nothing)
for λ_ in range(1,10):
    λ_/= 10
    #Iterate over each type of strat and compute statistics
    for strat_ in ["L", "S", "L-S"]:
        Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"] = LagSent_date_company.apply(lambda x: get_ric_pos_dict(strat_,n_companies,x.name,x,[Ret_date_company])  , axis=1)  
        Return_by_Strat_date.sort_index(inplace=True)
        Return_by_Strat_date[strat_+"("+str(n_companies)+")NCompanies[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x:  {"L":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"].values() if v == 1),"S":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"].values() if v == -1)} , axis=1)
        Return_by_Strat_date[strat_+"("+str(n_companies)+")RET_List[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"], Ret_date_company), axis=1)
        Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x: get_universe_weight_dict(unique_rics, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"], None),  axis=1)    
        
        Return_by_Strat_date[strat_+"("+str(n_companies)+")Optimal_Weight_Universe[λ="+str(λ_)+"]"] = getFULL_optimal_universe_weight_dict(Return_by_Strat_date,n_companies,   strat_,start_type_, λ_) 

        Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_List[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x: [x[strat_+"("+str(n_companies)+")Optimal_Weight_Universe[λ="+str(λ_)+"]"][key] for key in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"]], axis=1)
        Return_by_Strat_date[strat_+"[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x: sum(ret * w for ret, w in zip(x[strat_+"("+str(n_companies)+")RET_List[λ="+str(λ_)+"]"], x[strat_+"("+str(n_companies)+")Weight_List[λ="+str(λ_)+"]"])),  axis=1)
        Return_by_Strat_date[strat_+"[λ="+str(λ_)+"]"+" Turnover"] = Return_by_Strat_date.apply(lambda x: get_daily_turnover(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Optimal_Weight_Universe[λ="+str(λ_)+"]"],Ret_date_company), axis=1)
        Return_by_Strat_date[strat_+" Cumulative[λ="+str(λ_)+"]"] =  (Return_by_Strat_date[strat_+"[λ="+str(λ_)+"]"]+1).cumprod()
        Return_by_Strat_date[strat_+" Costs[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x: get_transaction_costs(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Optimal_Weight_Universe[λ="+str(λ_)+"]"]), axis=1)
        Return_by_Strat_date[strat_+" Net[λ="+str(λ_)+"]"] =  Return_by_Strat_date[strat_+"[λ="+str(λ_)+"]"] - Return_by_Strat_date[strat_+" Costs[λ="+str(λ_)+"]"]

In [ ]:
#Data with info for Stats
Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),Invest_FullData[["DateAdj",'MktCap(t-1)','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']], how="inner", on="DateAdj")
print("Data Final", len(Portfolio_Stats_Data), "Data Port", len(Return_by_Strat_date), "Data Baseline", len(Invest_FullData))
display(Portfolio_Stats_Data.head(2))

In [ ]:
data_check = []
#Strat Stats
for strat in ['L', 'S', 'L-S']:
    strat_data = pd.DataFrame()
    for λ_ in range(1,10):
        λ_/= 10
        strat_ = strat+"[λ="+str(λ_)+"]"
        print( "\t Strat: ",strat_)    # name to show, column of returns, column of net returns, dataframe
        df = paper_stats_table_Costs(strat_, strat_, strat+" Net[λ="+str(λ_)+"]",  Portfolio_Stats_Data)
        df.loc[strat_, "Turnover (%)"] = np.mean(Portfolio_Stats_Data[strat_+" Turnover"].iloc[1:])*100
        display(df)
        col_list = list(df.columns)
        for col in col_list:
            for index, row in df.iterrows():
                index_name = index if index[:index.find("[")] not in  ['L', 'S', 'L-S'] else ""
                strat_data.loc[strat_,f"{col}{index_name}"] = df.loc[index,col]
    data_check.append(strat_data)
    #Save stats
    # Position the dataframes in the worksheet.
    strat_data.to_excel(Cost_writer, sheet_name=str(n_companies)+'OP_Costs_SentStart>'+strat,startrow=1)

### Start EW

In [ ]:
#Load Saved Dataframe with Daily Sentiment classifications
with open("5)InvestmentStrat/"+file_name, "rb") as fp:  
    Invest_FullData_Limited = pickle.load(fp)
# Separate Data
Ret_date_company = Invest_FullData_Limited["Ret"]
LagSent_date_company = Invest_FullData_Limited["LagSent"]
LagMktCap_date_company = Invest_FullData_Limited["LagMktCap"]
Invest_FullData = Invest_FullData_Limited["InvestData"]

In [ ]:
#Construct List of top and bottom companies for portfolio creation  
#(auxiliary)
n_companies = 100
unique_rics = list(LagSent_date_company.columns)
start_type_ = ["current",unique_rics,True]   # first element can be  "current"   or "random"
Return_by_Strat_date = pd.DataFrame()  
# Loop over different lambdas (percentage of positions to change) max 1 (= change everthing) and min 0 (=change nothing)
for λ_ in range(1,10):
    λ_/= 10
    #Iterate over each type of strat and compute statistics
    for strat_ in ["L", "S", "L-S"]:
        Return_by_Strat_date[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"] = LagSent_date_company.apply(lambda x: get_ric_pos_dict(strat_,n_companies,x.name,x,[Ret_date_company])  , axis=1)  
        Return_by_Strat_date.sort_index(inplace=True)
        Return_by_Strat_date[strat_+"("+str(n_companies)+")NCompanies[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x:  {"L":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"].values() if v == 1),"S":sum(1 for v in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"].values() if v == -1)} , axis=1)
        Return_by_Strat_date[strat_+"("+str(n_companies)+")RET_List[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x: get_ric_value_dict(x.name, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"], Ret_date_company), axis=1)
        Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_Universe[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x: get_universe_weight_dict(unique_rics, x[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"], None),  axis=1)    
                
        Return_by_Strat_date[strat_+"("+str(n_companies)+")Optimal_Weight_Universe[λ="+str(λ_)+"]"] = getFULL_optimal_universe_weight_dict(Return_by_Strat_date,n_companies,   strat_,start_type_, λ_) 

        Return_by_Strat_date[strat_+"("+str(n_companies)+")Weight_List[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x: [x[strat_+"("+str(n_companies)+")Optimal_Weight_Universe[λ="+str(λ_)+"]"][key] for key in x[strat_+"("+str(n_companies)+")RIC_Pos_Dict[λ="+str(λ_)+"]"]], axis=1)
        Return_by_Strat_date[strat_+"[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x: sum(ret * w for ret, w in zip(x[strat_+"("+str(n_companies)+")RET_List[λ="+str(λ_)+"]"], x[strat_+"("+str(n_companies)+")Weight_List[λ="+str(λ_)+"]"])),  axis=1)
        Return_by_Strat_date[strat_+" Cumulative[λ="+str(λ_)+"]"] =  (Return_by_Strat_date[strat_+"[λ="+str(λ_)+"]"]+1).cumprod()
        Return_by_Strat_date[strat_+" Costs[λ="+str(λ_)+"]"] = Return_by_Strat_date.apply(lambda x: get_transaction_costs(x.name, Return_by_Strat_date[strat_+"("+str(n_companies)+")Optimal_Weight_Universe[λ="+str(λ_)+"]"]), axis=1)
        Return_by_Strat_date[strat_+" Net[λ="+str(λ_)+"]"] =  Return_by_Strat_date[strat_+"[λ="+str(λ_)+"]"] - Return_by_Strat_date[strat_+" Costs[λ="+str(λ_)+"]"]

In [ ]:
#Data with info for Stats
Portfolio_Stats_Data = pd.merge(Return_by_Strat_date.reset_index(),Invest_FullData[["DateAdj",'MktCap(t-1)','Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF','Mom']], how="inner", on="DateAdj")
print("Data Final", len(Portfolio_Stats_Data), "Data Port", len(Return_by_Strat_date), "Data Baseline", len(Invest_FullData))
display(Portfolio_Stats_Data.head(2))

In [ ]:
#Strat Stats
for strat in ['L', 'S', 'L-S']:
    strat_data = pd.DataFrame()
    for λ_ in range(1,10):
        λ_/= 10
        strat_ = strat+"[λ="+str(λ_)+"]"
        print( "\t Strat: ",strat_)    # name to show, column of returns, column of net returns, dataframe
        df = paper_stats_table_Costs(strat_, strat_, strat+" Net[λ="+str(λ_)+"]",  Portfolio_Stats_Data)
        display(df)
        col_list = list(df.columns)
        for col in col_list:
            for index, row in df.iterrows():
                index_name = index if index[:index.find("[")] not in  ['L', 'S', 'L-S'] else ""
                strat_data.loc[strat_,f"{col}{index_name}"] = df.loc[index,col]
    #Save stats
    # Position the dataframes in the worksheet.
    strat_data.to_excel(Cost_writer, sheet_name=str(n_companies)+'OP_Costs_EWStart>'+strat,startrow=1)

# Investment Strategy Testing Based on Other Paper Ideas


<u> **1**) Strategies based on daily `relative` classification [3 types just long(**L**), just short(**S**) and long-short(**L-S**]:</u>
> **Relative_Strat_1>** Each day go long(short) on top(bottom) 20% ranked stocks (=100 stocks) if there are signals for that amount. Meaning startegy can include less than 100 stocks but not more. Signal is based purely on previous day Sentiment (**1 var**)
> **Relative_Strat_2>** Each day go long(short) on top(bottom) 20% ranked stocks (=100 stocks) if there are signal for that amount. Meaning startegy can include less than 100 stocks but not more. Signal is based on the alignment of Previous day sentiment, sentiment shock and sentiment trend. (**3 var**)
> **Relative_Strat_2>** Each day go long(short) on top(bottom) 20% ranked stocks (=100 stocks) if there are signal for that amount. Meaning startegy can include less than 100 stocks but not more. Signal is based on the alignment of sentiment shock and sentimetn trend. (**2 var**)

<u> **2**) Strategies based on daily `absolute` classification [3 types just long(**L**), just short(**S**) and long-short(**L-S**]:</u>
> **Absolute_Strat_1>** Pick top winners (losers) (capped at 100) based on previous day sentiment (**1 var**)

<u> **2**) Strategies Adjusted Less Frequently [3 types just long(**L**), just short(**S**) and long-short(**L-S**]:</u>
> **Passive_Strat_1>** 


**Key** assumptions:
> **Transaction Costs>** 20bp for round trip (average between Tetlock and News Based Traiding Reference Paper)
> **No signal** assumption: If no signal is given then get risk free

### Functions

In [ ]:
def stats_table(Strat_Returns):
    #Main Metrics
    Strat_Statistics.loc["Count"] = len(Strat_Returns)
    Strat_Statistics.loc[""] = ""
    Strat_Statistics.loc["Mean %"] = np.average(Strat_Returns)*100*12
    Strat_Statistics.loc["(mean) t-stat"] = np.average(Strat_Returns) / (np.std(Strat_Returns)/ (Strat_Statistics.loc["Count"].values[0]**0.5)) 
    Strat_Statistics.loc["(mean) p-value"] = (1-norm.cdf(Strat_Statistics.loc["(mean) t-stat"].values[0]))*2
    Strat_Statistics.loc["-"] = "-"
    Strat_Statistics.loc["Std. Dev. (%)"] = np.round(np.std(Strat_Returns)*100*(12**0.5),2)
    Strat_Statistics.loc["--"] = "--"
    Strat_Statistics.loc["Sharpe ratio"] = Strat_Statistics.loc["Mean %"]/ Strat_Statistics.loc["Std. Dev. (%)"]
    Strat_Statistics.loc["(SR) t-stat"] = (Strat_Statistics.loc["Sharpe ratio"].values[0]/(12**0.5))/np.sqrt((1+0.5*(Strat_Statistics.loc["Sharpe ratio"].values[0]/(12**0.5))**2)/Strat_Statistics.loc["Count"].values[0])
    Strat_Statistics.loc["(SR) p-value"] = (1-norm.cdf(Strat_Statistics.loc["(SR) t-stat"].values[0]))*2
    Strat_Statistics.loc["---"] = "---"
    Strat_Statistics.loc["Skewness"] = skew(Strat_Returns)
    Strat_Statistics.loc["Kurtosis"] = kurtosis(Strat_Returns)
    Strat_Statistics.loc["----"] = "----"
    Strat_Statistics.loc["JB test statistic"] = (Strat_Statistics.loc["Count"].values[0]/6)*(Strat_Statistics.loc["Skewness"].values[0]**2+0.25*(Strat_Statistics.loc["Kurtosis"].values[0]**0.5))
    Strat_Statistics.loc["(JB) p-value"] = 1-chi2.cdf(Strat_Statistics.loc["JB test statistic"].values[0], 2)
    Strat_Statistics.loc["-----"] = "-----"
    Strat_Statistics.loc["Minimum (%)"] = Strat_Returns.min()
    Strat_Statistics.loc["Percentile 25 (%)"] = np.percentile(Strat_Returns, 25)
    Strat_Statistics.loc["Median (%)"] = Strat_Returns.median()
    Strat_Statistics.loc["Percentile 75 (%)"] = np.percentile(Strat_Returns, 75)
    Strat_Statistics.loc["Maximum (%)"] = Strat_Returns.max()
    Strat_Statistics.loc["------"] = "------"
    Strat_Statistics.loc["AR(1) (%)"] = Strat_Returns.corr(Strat_Returns.shift(1))
    Strat_Statistics.loc["p-value"] = 2*(1-norm.cdf(Strat_Statistics.loc["Count"].values[0]**0.5*np.abs(Strat_Statistics.loc["AR(1) (%)"].values[0])))
    return Strat_Statistics

In [ ]:
def FamaFrench(data, y_var):
    X_Fama3Factors = data[['Mkt-RF', 'SMB', 'HML']]
    Y_ExcessReturns = data[y_var] - data['RF']
    X = sm.add_constant(X_Fama3Factors)
    ff_model = sm.OLS(Y_ExcessReturns, X).fit()
    print(ff_model.summary())
    intercept, b1, b2, b3 = ff_model.params
    return intercept, b1, b2, b3

## Relative and Absolute Strats

### (1) Relative Strategies

In [ ]:
def decile(x,n_deciles):
    try:
        return pd.qcut(x, n_deciles, labels=False)
    except Exception as e:
        if x.isnull().all():
            return x
        print(e,x)

In [ ]:
def returns_adj_transaction_costs(strat_row,strat_data,sinal_type,cost):
    date = strat_row.name
    strat_date_iloc = strat_data.index.get_loc(date)
    current_company_list = strat_data.iloc[strat_date_iloc][sinal_type+"RIC_List"] 
    
    #Dealing with cases where no ranking provided
    if len(current_company_list)==0:
        return pd.Series([0,0], index=[sinal_type+"NTransactions", sinal_type+"Cost"])
    else:
        #(exception of first case where all positions have costs)
        if strat_date_iloc==0:
            n_transactions = len(current_company_list)
            return pd.Series([n_transactions,cost], index=[sinal_type+"NTransactions", sinal_type+"Cost"])
        else:
            old_company_list = strat_data.iloc[strat_date_iloc-1][sinal_type+"RIC_List"]
            n_unchanged_companies = len(list(set(old_company_list).intersection(current_company_list)))
            n_transactions = (len(current_company_list)-n_unchanged_companies)
            costs = n_transactions*cost/len(current_company_list) 
            return pd.Series([n_transactions,costs], index=[sinal_type+"NTransactions", sinal_type+"Cost"]) 

#### Relative_Strat_1
(Just Previous Day Sentiment Variable)

*For 100 (200 & 50) companies*
>With Costs
**Best Performers**: Long Strategy is profitable BUT Everything performs badly, highly driven by transaction costs.
**Notes**: Not even reversal intuition can help to achieve good perforance

>Without  Costs
**Best Performers**: Long strategy is similar to benchmark, others result in negative returns. The opposite of short outperforms benchmark (see notes for hypothesis).
**Notes**: It appears that negative sentiment creates panic but then bounces back within the next day so buying becomes more profitable (supports market reversal)


In [ ]:
    #Return by date and company
Ret_date_company = Invest_FullData.pivot(index='Date', columns='RIC', values='RET')    
Ret_date_company = Ret_date_company.iloc[1:]
    #Lagged sent by date and company
LagSent_date_company = Invest_FullData.pivot(index='Date', columns='RIC', values='Avg_Confidence(t-1)') 
LagSent_date_company = LagSent_date_company.iloc[1:]

    #safety check
dates_not_common = set(LagSent_date_company.index).symmetric_difference(set(Ret_date_company.index))
print(f"The following dates are not available on both sides: {dates_not_common}")
      
    #Get Ranking of only negative signals (1st ranking = lowest sentiment | Lower number = bigger negative signal)
NEG_LagSent_date_company = LagSent_date_company.copy()
NEG_LagSent_date_company[NEG_LagSent_date_company>=0.5] = np.nan
NEG_RANK_date_company_LagSent = NEG_LagSent_date_company.apply(lambda x: x.rank(method='first', ascending=True), axis=1) 
    #Get Ranking of only negative signals (1st ranking = highest sentiment | Lower number = bigger positive signal)
POS_LagSent_date_company = LagSent_date_company.copy()
POS_LagSent_date_company[POS_LagSent_date_company<=0.5] = np.nan
POS_RANK_date_company_LagSent = POS_LagSent_date_company.apply(lambda x: x.rank(method='first', ascending=False), axis=1) 
      
    #Construct lists of companies according to top and bottom signals
        #(auxiliary)
n_companies = 200
Return_by_Strat_date = pd.DataFrame()
    #(companies list, Nº of companies, Average return, N of transactions, Average Costs)
for df, sinal_type in zip([NEG_RANK_date_company_LagSent,POS_RANK_date_company_LagSent],["Neg("+str(n_companies)+")","Pos("+str(n_companies)+")"]):
        Return_by_Strat_date[sinal_type+"RIC_List"] = df.apply(lambda x: list(x[x<=n_companies].index), axis=1)
        Return_by_Strat_date[sinal_type+"NCompanies"] = Return_by_Strat_date.apply(lambda x: len(x[sinal_type+"RIC_List"]), axis=1)
        Return_by_Strat_date[sinal_type+"Avg_Ret"] = Return_by_Strat_date.apply(lambda x: np.nanmean(Ret_date_company.loc[x.name][Ret_date_company.loc[x.name].index.isin(x[sinal_type+"RIC_List"])]), axis=1)
        Return_by_Strat_date[[sinal_type+"NTransactions", sinal_type+"Cost"]] =  Return_by_Strat_date.apply(lambda x: returns_adj_transaction_costs(x,Return_by_Strat_date,sinal_type,transaction_cost) , axis=1)

# Strategy returns
    #(daily return)
Return_by_Strat_date["L"] = (1) *Return_by_Strat_date["Pos("+str(n_companies)+")Avg_Ret"] - Return_by_Strat_date["Pos("+str(n_companies)+")Cost"]
Return_by_Strat_date["S"] = (-1) * Return_by_Strat_date["Neg("+str(n_companies)+")Avg_Ret"] - Return_by_Strat_date["Neg("+str(n_companies)+")Cost"]
Return_by_Strat_date["L-S"] = ((1)*Return_by_Strat_date["Pos("+str(n_companies)+")Avg_Ret"] + (-1)*Return_by_Strat_date["Neg("+str(n_companies)+")Avg_Ret"]) -(Return_by_Strat_date["Pos("+str(n_companies)+")Cost"]+Return_by_Strat_date["Neg("+str(n_companies)+")Cost"]) 
    #(cumulative return)
Return_by_Strat_date.sort_index(inplace=True)
Return_by_Strat_date["L Cumulative"] =  (Return_by_Strat_date["L"]+1).cumprod()
Return_by_Strat_date["S Cumulative"] = (Return_by_Strat_date["S"]+1).cumprod()
Return_by_Strat_date["L-S Cumulative"] = (Return_by_Strat_date["L-S"]+1).cumprod()
Return_by_Strat_date.head(2)

In [ ]:
#Baseline
baseline = Buy_Hold[Buy_Hold["Date"].isin(Return_by_Strat_date.index)]
plt.plot(baseline["Date"],baseline["Cumulative"], label = "Buy & Hold", color='tab:red')
#Strategy
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index, strat in enumerate([col for col in Return_by_Strat_date if "Cumulative" in col]):
    plt.plot(Return_by_Strat_date.index,Return_by_Strat_date[strat], label = strat, color='tab:'+color[index])
plt.xlabel('Date')
plt.title("Relative strategy Based on Average_Set")
plt.legend()
plt.show()

In [ ]:
#Statistics Table
statistics_table = pd.DataFrame()
for column in ["L","S","L-S"]:
    statistics_table[column] = stats_table(Return_by_Strat_date[column])
display(statistics_table)

In [ ]:
#Relevant data + Fama French
StratReturn_and_Fama = pd.merge(Return_by_Strat_date.reset_index(),FF3_Data, how="left", on=['Date'])
FamaFrench(StratReturn_and_Fama, "L")

#### Relative_Strat_2
(Previous Day Sent and Sentiment Shock and Trend Variables as signals)

>With Costs
**Best Performers**: Both Long and short have very similar performance, being positive despote the transaction costs. However they are below the Benchmark.
**Notes**: 

>Without  Costs
**Best Performers**: (With 7 days based shock and trend) Both Long and Short Startegies outpereform benchmark

In [ ]:
    #Return by date and company
Ret_date_company = Invest_FullData.pivot(index='Date', columns='RIC', values='RET')    
Ret_date_company = Ret_date_company.iloc[1:]
    #Lagged sent by date and company
LagSent_date_company = Invest_FullData.pivot(index='Date', columns='RIC', values='Avg_Confidence(t-1)') 
LagSent_date_company = LagSent_date_company.iloc[1:]
    #Lagged sent shock by date and company
shock_var = [col for col in Invest_FullData.columns if "_Shock" in col][0]
LagSentShock_date_company = Invest_FullData.pivot(index='Date', columns='RIC', values=shock_var) 
LagSentShock_date_company = LagSentShock_date_company.iloc[1:]
    #Lagged sent Trend by date and company
trend_var = [col for col in Invest_FullData.columns if "_Trend" in col][0]
LagSentTrend_date_company = Invest_FullData.pivot(index='Date', columns='RIC', values=trend_var) 
LagSentTrend_date_company = LagSentTrend_date_company.iloc[1:]

    #safety check
dates_not_common = set(LagSent_date_company.index).symmetric_difference(set(Ret_date_company.index))
print(f"The following dates are not available on both sides: {dates_not_common}")
    
# Previous DaY Sentiment
    #Get Ranking of only negative signals (1st ranking = lowest sentiment | Lower number = bigger negative signal)
NEG_LagSent_date_company = LagSent_date_company.copy()
NEG_LagSent_date_company[NEG_LagSent_date_company>=0.5] = np.nan
NEG_RANK_date_company_LagSent = NEG_LagSent_date_company.apply(lambda x: x.rank(method='first', ascending=True), axis=1) 
    #Get Ranking of only negative signals (1st ranking = highest sentiment | Lower number = bigger positive signal)
POS_LagSent_date_company = LagSent_date_company.copy()
POS_LagSent_date_company[POS_LagSent_date_company<=0.5] = np.nan
POS_RANK_date_company_LagSent = POS_LagSent_date_company.apply(lambda x: x.rank(method='first', ascending=False), axis=1) 

# Schock Sentiment
    #Get Ranking of only negative signals (1st ranking = lowest sentiment | Lower number = bigger negative signal)
NEG_LagSentShock_date_company = LagSentShock_date_company.copy()
NEG_LagSentShock_date_company[NEG_LagSentShock_date_company>=0] = np.nan
NEG_RANK_date_company_LagSentShock = NEG_LagSentShock_date_company.apply(lambda x: x.rank(method='first', ascending=True), axis=1) 
    #Get Ranking of only negative signals (1st ranking = highest sentiment | Lower number = bigger positive signal)
POS_LagSentShock_company = LagSentShock_date_company.copy()
POS_LagSentShock_company[POS_LagSentShock_company<=0] = np.nan
POS_RANK_date_company_LagSentShock = POS_LagSentShock_company.apply(lambda x: x.rank(method='first', ascending=False), axis=1) 

# Trend Sentiment
    #Get Ranking of only negative signals (1st ranking = lowest sentiment | Lower number = bigger negative signal)
NEG_LagSentTrend_date_company = LagSentTrend_date_company.copy()
NEG_LagSentTrend_date_company[NEG_LagSentTrend_date_company>=0] = np.nan
NEG_RANK_date_company_LagSentTrend = NEG_LagSentTrend_date_company.apply(lambda x: x.rank(method='first', ascending=True), axis=1) 
    #Get Ranking of only negative signals (1st ranking = highest sentiment | Lower number = bigger positive signal)
POS_LagSentTrend_company = LagSentTrend_date_company.copy()
POS_LagSentTrend_company[POS_LagSentTrend_company<=0] = np.nan
POS_RANK_date_company_LagSentTrend = POS_LagSentTrend_company.apply(lambda x: x.rank(method='first', ascending=False), axis=1) 

    #Joined Ranking (based on mean of all rankings)
NegRANK_date_company = pd.concat([NEG_RANK_date_company_LagSent,NEG_RANK_date_company_LagSentShock,NEG_RANK_date_company_LagSentTrend]).groupby("Date").agg(np.nanmean)
POSRANK_date_company = pd.concat([POS_RANK_date_company_LagSent,POS_RANK_date_company_LagSentShock,POS_RANK_date_company_LagSentTrend]).groupby("Date").agg(np.nanmean)


    #Construct lists of companies according to top and bottom signals
        #(auxiliary)
n_companies = 200
Return_by_Strat_date = pd.DataFrame()
    #(companies list, Nº of companies, Average return, N of transactions, Average Costs)
for df, sinal_type in zip([NegRANK_date_company,POSRANK_date_company],["Neg("+str(n_companies)+")","Pos("+str(n_companies)+")"]):
        Return_by_Strat_date[sinal_type+"RIC_List"] = df.apply(lambda x: list(x[x<=n_companies].index), axis=1)
        Return_by_Strat_date[sinal_type+"NCompanies"] = Return_by_Strat_date.apply(lambda x: len(x[sinal_type+"RIC_List"]), axis=1)
        Return_by_Strat_date[sinal_type+"Avg_Ret"] = Return_by_Strat_date.apply(lambda x: np.nanmean(Ret_date_company.loc[x.name][Ret_date_company.loc[x.name].index.isin(x[sinal_type+"RIC_List"])]), axis=1)
        Return_by_Strat_date[[sinal_type+"NTransactions", sinal_type+"Cost"]] =  Return_by_Strat_date.apply(lambda x: returns_adj_transaction_costs(x,Return_by_Strat_date,sinal_type,transaction_cost) , axis=1)

# Strategy returns
    #(daily return)
Return_by_Strat_date["L"] = (1) *Return_by_Strat_date["Pos("+str(n_companies)+")Avg_Ret"] - Return_by_Strat_date["Pos("+str(n_companies)+")Cost"]
Return_by_Strat_date["S"] = (1) * Return_by_Strat_date["Neg("+str(n_companies)+")Avg_Ret"] - Return_by_Strat_date["Neg("+str(n_companies)+")Cost"]
Return_by_Strat_date["L-S"] = ((1)*Return_by_Strat_date["Pos("+str(n_companies)+")Avg_Ret"] + (-1)*Return_by_Strat_date["Neg("+str(n_companies)+")Avg_Ret"]) -(Return_by_Strat_date["Pos("+str(n_companies)+")Cost"]+Return_by_Strat_date["Neg("+str(n_companies)+")Cost"]) 
    #(cumulative return)
Return_by_Strat_date.sort_index(inplace=True)
Return_by_Strat_date["L Cumulative"] =  (Return_by_Strat_date["L"]+1).cumprod()
Return_by_Strat_date["S Cumulative"] = (Return_by_Strat_date["S"]+1).cumprod()
Return_by_Strat_date["L-S Cumulative"] = (Return_by_Strat_date["L-S"]+1).cumprod()
Return_by_Strat_date.head(2)

In [ ]:
#Baseline
baseline = Buy_Hold[Buy_Hold["Date"].isin(Return_by_Strat_date.index)]
plt.plot(baseline["Date"],baseline["Cumulative"], label = "Buy & Hold", color='tab:red')
#Strategy
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index, strat in enumerate([col for col in Return_by_Strat_date if "Cumulative" in col]):
    plt.plot(Return_by_Strat_date.index,Return_by_Strat_date[strat], label = strat, color='tab:'+color[index])
plt.xlabel('Date')
plt.title("Relative strategy Based on Average_Set")
plt.legend()
plt.show()

In [ ]:
#Statistics Table
statistics_table = pd.DataFrame()
for column in ["L","S","L-S"]:
    statistics_table[column] = stats_table(Return_by_Strat_date[column])
display(statistics_table)

In [ ]:
#Relevant data + Fama French
StratReturn_and_Fama = pd.merge(Return_by_Strat_date.reset_index(),FF3_Data, how="left", on=['Date'])
FamaFrench(StratReturn_and_Fama, "S")

#### Relative_Strat_3
(Sentiment Shock and Trend Variables as signals)

>With Costs
**Best Performers**: 
**Notes**: 

>Without  Costs
**Best Performers**:

In [ ]:
    #Return by date and company
Ret_date_company = Invest_FullData.pivot(index='Date', columns='RIC', values='RET')    
Ret_date_company = Ret_date_company.iloc[1:]
    #Lagged sent shock by date and company
shock_var = [col for col in Invest_FullData.columns if "_Shock" in col][0]
LagSentShock_date_company = Invest_FullData.pivot(index='Date', columns='RIC', values=shock_var) 
LagSentShock_date_company = LagSentShock_date_company.iloc[1:]
    #Lagged sent Trend by date and company
trend_var = [col for col in Invest_FullData.columns if "_Trend" in col][0]
LagSentTrend_date_company = Invest_FullData.pivot(index='Date', columns='RIC', values=trend_var) 
LagSentTrend_date_company = LagSentTrend_date_company.iloc[1:]

    #safety check
dates_not_common = set(LagSent_date_company.index).symmetric_difference(set(Ret_date_company.index))
print(f"The following dates are not available on both sides: {dates_not_common}")


# Schock Sentiment
    #Get Ranking of only negative signals (1st ranking = lowest sentiment | Lower number = bigger negative signal)
NEG_LagSentShock_date_company = LagSentShock_date_company.copy()
NEG_LagSentShock_date_company[NEG_LagSentShock_date_company>=0] = np.nan
NEG_RANK_date_company_LagSentShock = NEG_LagSentShock_date_company.apply(lambda x: x.rank(method='first', ascending=True), axis=1) 
    #Get Ranking of only negative signals (1st ranking = highest sentiment | Lower number = bigger positive signal)
POS_LagSentShock_company = LagSentShock_date_company.copy()
POS_LagSentShock_company[POS_LagSentShock_company<=0] = np.nan
POS_RANK_date_company_LagSentShock = POS_LagSentShock_company.apply(lambda x: x.rank(method='first', ascending=False), axis=1) 

# Trend Sentiment
    #Get Ranking of only negative signals (1st ranking = lowest sentiment | Lower number = bigger negative signal)
NEG_LagSentTrend_date_company = LagSentTrend_date_company.copy()
NEG_LagSentTrend_date_company[NEG_LagSentTrend_date_company>=0] = np.nan
NEG_RANK_date_company_LagSentTrend = NEG_LagSentTrend_date_company.apply(lambda x: x.rank(method='first', ascending=True), axis=1) 
    #Get Ranking of only negative signals (1st ranking = highest sentiment | Lower number = bigger positive signal)
POS_LagSentTrend_company = LagSentTrend_date_company.copy()
POS_LagSentTrend_company[POS_LagSentTrend_company<=0] = np.nan
POS_RANK_date_company_LagSentTrend = POS_LagSentTrend_company.apply(lambda x: x.rank(method='first', ascending=False), axis=1) 

    #Joined Ranking (based on mean of all rankings)
NegRANK_date_company = pd.concat([NEG_RANK_date_company_LagSentShock,NEG_RANK_date_company_LagSentTrend]).groupby("Date").agg(np.nanmean)
POSRANK_date_company = pd.concat([POS_RANK_date_company_LagSentShock,POS_RANK_date_company_LagSentTrend]).groupby("Date").agg(np.nanmean)


    #Construct lists of companies according to top and bottom signals
        #(auxiliary)
n_companies = 200
Return_by_Strat_date = pd.DataFrame()
    #(companies list, Nº of companies, Average return, N of transactions, Average Costs)
for df, sinal_type in zip([NegRANK_date_company,POSRANK_date_company],["Neg("+str(n_companies)+")","Pos("+str(n_companies)+")"]):
        Return_by_Strat_date[sinal_type+"RIC_List"] = df.apply(lambda x: list(x[x<=n_companies].index), axis=1)
        Return_by_Strat_date[sinal_type+"NCompanies"] = Return_by_Strat_date.apply(lambda x: len(x[sinal_type+"RIC_List"]), axis=1)
        Return_by_Strat_date[sinal_type+"Avg_Ret"] = Return_by_Strat_date.apply(lambda x: np.nanmean(Ret_date_company.loc[x.name][Ret_date_company.loc[x.name].index.isin(x[sinal_type+"RIC_List"])]), axis=1)
        Return_by_Strat_date[[sinal_type+"NTransactions", sinal_type+"Cost"]] =  Return_by_Strat_date.apply(lambda x: returns_adj_transaction_costs(x,Return_by_Strat_date,sinal_type,transaction_cost) , axis=1)

# Strategy returns
    #(daily return)
Return_by_Strat_date["L"] = (1) *Return_by_Strat_date["Pos("+str(n_companies)+")Avg_Ret"] - Return_by_Strat_date["Pos("+str(n_companies)+")Cost"]
Return_by_Strat_date["S"] = (1) * Return_by_Strat_date["Neg("+str(n_companies)+")Avg_Ret"] - Return_by_Strat_date["Neg("+str(n_companies)+")Cost"]
Return_by_Strat_date["L-S"] = ((1)*Return_by_Strat_date["Pos("+str(n_companies)+")Avg_Ret"] + (-1)*Return_by_Strat_date["Neg("+str(n_companies)+")Avg_Ret"]) -(Return_by_Strat_date["Pos("+str(n_companies)+")Cost"]+Return_by_Strat_date["Neg("+str(n_companies)+")Cost"]) 
    #(cumulative return)
Return_by_Strat_date.sort_index(inplace=True)
Return_by_Strat_date["L Cumulative"] =  (Return_by_Strat_date["L"]+1).cumprod()
Return_by_Strat_date["S Cumulative"] = (Return_by_Strat_date["S"]+1).cumprod()
Return_by_Strat_date["L-S Cumulative"] = (Return_by_Strat_date["L-S"]+1).cumprod()
Return_by_Strat_date.head(2)

In [ ]:
#Baseline
baseline = Buy_Hold[Buy_Hold["Date"].isin(Return_by_Strat_date.index)]
plt.plot(baseline["Date"],baseline["Cumulative"], label = "Buy & Hold", color='tab:red')
#Strategy
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index, strat in enumerate([col for col in Return_by_Strat_date if "Cumulative" in col]):
    plt.plot(Return_by_Strat_date.index,Return_by_Strat_date[strat], label = strat, color='tab:'+color[index])
plt.xlabel('Date')
plt.title("Relative strategy Based on Average_Set")
plt.legend()
plt.show()

In [ ]:
#Statistics Table
statistics_table = pd.DataFrame()
for column in ["L","S","L-S"]:
    statistics_table[column] = stats_table(Return_by_Strat_date[column])
display(statistics_table)

In [ ]:
#Relevant data + Fama French
StratReturn_and_Fama = pd.merge(Return_by_Strat_date.reset_index(),FF3_Data, how="left", on=['Date'])
FamaFrench(StratReturn_and_Fama, "S")

### (2) Absolute Strategies

#### Absolute_Strat_1
(Just Previous Day Sentiment Variable)

*For 100 companies*
>With Costs
**Best Performers**: All perform BAd. However short startegy starts with a relatively good performance

>Without  Costs
**Best Performers**: Long strategy, very similar to benchmark. The reversal of short startegy has best results.
**Notes**: Appear to be much more volatile than the others.

In [ ]:
    #Return by date and company
Ret_date_company = Invest_FullData.pivot(index='Date', columns='RIC', values='RET')    
Ret_date_company = Ret_date_company.iloc[1:]
    #Lagged sent by date and company
LagSent_date_company = Invest_FullData.pivot(index='Date', columns='RIC', values='Avg_Confidence(t-1)') 
LagSent_date_company = LagSent_date_company.iloc[1:]

    #safety check
dates_not_common = set(LagSent_date_company.index).symmetric_difference(set(Ret_date_company.index))
print(f"The following dates are not available on both sides: {dates_not_common}")
      
    #Get Ranking of only top negative signals (1st ranking = lowest sentiment | Lower number = bigger negative signal)
neg_threshold = 0.2
NEG_LagSent_date_company = LagSent_date_company.copy()
NEG_LagSent_date_company[NEG_LagSent_date_company>neg_threshold] = np.nan
NEG_RANK_date_company_LagSent = NEG_LagSent_date_company.apply(lambda x: x.rank(method='first', ascending=True), axis=1) 
    #Get Ranking of only top negative signals (1st ranking = highest sentiment | Lower number = bigger positive signal)
pos_threshold = 0.80
POS_LagSent_date_company = LagSent_date_company.copy()
POS_LagSent_date_company[POS_LagSent_date_company<pos_threshold] = np.nan
POS_RANK_date_company_LagSent = POS_LagSent_date_company.apply(lambda x: x.rank(method='first', ascending=False), axis=1) 
      
    #Construct lists of companies according to top and bottom signals
        #(auxiliary)
n_companies = 50
Return_by_Strat_date = pd.DataFrame()
    #(companies list, Nº of companies, Average return, N of transactions, Average Costs)
for df, sinal_type in zip([NEG_RANK_date_company_LagSent,POS_RANK_date_company_LagSent],["Neg("+str(n_companies)+")","Pos("+str(n_companies)+")"]):
        Return_by_Strat_date[sinal_type+"RIC_List"] = df.apply(lambda x: list(x[x<=n_companies].index), axis=1)
        Return_by_Strat_date[sinal_type+"NCompanies"] = Return_by_Strat_date.apply(lambda x: len(x[sinal_type+"RIC_List"]), axis=1)
        Return_by_Strat_date[sinal_type+"Avg_Ret"] = Return_by_Strat_date.apply(lambda x: np.nanmean(Ret_date_company.loc[x.name][Ret_date_company.loc[x.name].index.isin(x[sinal_type+"RIC_List"])]), axis=1)
        Return_by_Strat_date[[sinal_type+"NTransactions", sinal_type+"Cost"]] =  Return_by_Strat_date.apply(lambda x: returns_adj_transaction_costs(x,Return_by_Strat_date,sinal_type,transaction_cost) , axis=1)
        #For no allocation put on risk free (for this time period is zero)
        Return_by_Strat_date[sinal_type+"Avg_Ret"].fillna(0, inplace=True)
        
# Strategy returns
    #(daily return)
Return_by_Strat_date["L"] = (1) *Return_by_Strat_date["Pos("+str(n_companies)+")Avg_Ret"] - Return_by_Strat_date["Pos("+str(n_companies)+")Cost"]
Return_by_Strat_date["S"] = (1) * Return_by_Strat_date["Neg("+str(n_companies)+")Avg_Ret"] - Return_by_Strat_date["Neg("+str(n_companies)+")Cost"]
Return_by_Strat_date["L-S"] = ((1)*Return_by_Strat_date["Pos("+str(n_companies)+")Avg_Ret"] + (-1)*Return_by_Strat_date["Neg("+str(n_companies)+")Avg_Ret"]) #-(Return_by_Strat_date["Pos("+str(n_companies)+")Cost"]+Return_by_Strat_date["Neg("+str(n_companies)+")Cost"]) 
    #(cumulative return)
Return_by_Strat_date.sort_index(inplace=True)
Return_by_Strat_date["L Cumulative"] =  (Return_by_Strat_date["L"]+1).cumprod()
Return_by_Strat_date["S Cumulative"] = (Return_by_Strat_date["S"]+1).cumprod()
Return_by_Strat_date["L-S Cumulative"] = (Return_by_Strat_date["L-S"]+1).cumprod()
Return_by_Strat_date.head(2)

In [ ]:
#Baseline
baseline = Buy_Hold[Buy_Hold["Date"].isin(Return_by_Strat_date.index)]
plt.plot(baseline["Date"],baseline["Cumulative"], label = "Buy & Hold", color='tab:red')
#Strategy
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index, strat in enumerate([col for col in Return_by_Strat_date if "Cumulative" in col]):
    plt.plot(Return_by_Strat_date.index,Return_by_Strat_date[strat], label = strat, color='tab:'+color[index])
plt.xlabel('Date')
plt.title("Relative strategy Based on Average_Set")
plt.legend()
plt.show()

In [ ]:
#Statistics Table
statistics_table = pd.DataFrame()
for column in ["L","S","L-S"]:
    statistics_table[column] = stats_table(Return_by_Strat_date[column])
display(statistics_table)

In [ ]:
#Relevant data + Fama French
StratReturn_and_Fama = pd.merge(Return_by_Strat_date.reset_index(),FF3_Data, how="left", on=['Date'])
FamaFrench(StratReturn_and_Fama, "L")

---------------

In [ ]:
    #Return by date and company
Ret_date_company = Invest_Data.pivot(index='Date', columns='RIC', values='RET')     

    #Lagged sent by date and company
LagSent_date_company = Invest_Data.pivot(index='Date', columns='RIC', values='Avg_Confidence(t-1)') 
LagSent_date_company.dropna(how="all", inplace=True)
#LagSent_date_company.fillna(0.5, inplace=True)  #0.5 is the proxy to neutral

    #Kepp only dates in common
relevant_dates = Ret_date_company.index.intersection(LagSent_date_company.index)
Ret_date_company = Ret_date_company.loc[relevant_dates]
LagSent_date_company = LagSent_date_company.loc[relevant_dates]

    #Average returns for companies > threshold
threshold_start = 0.8
Return_long_threshold = pd.DataFrame()
for t in range(int(threshold_start*100),100,5):
        threshold = t/100
        Return_long_threshold["Long>="+str(threshold)] =  Ret_date_company.apply(lambda x: np.average(x[x.index.isin(list(LagSent_date_company.loc[x.name][LagSent_date_company.loc[x.name]>=threshold].index))]) , axis=1)
        Return_long_threshold.sort_index(inplace=True)
        Return_long_threshold["Long>="+str(threshold) + " Cumulative"] = (Return_long_threshold["Long>="+str(threshold)]+1).cumprod()

In [ ]:
#Statistics Table
statistics_table = pd.DataFrame()
for column in [col for col in Return_long_threshold.columns if ("Cumulative" not in col)]:
    statistics_table[column] = stats_table(Return_long_threshold[column])
display(statistics_table)

In [ ]:
#Baseline
baseline = Buy_Hold[Buy_Hold["Date"].isin(Return_by_decile_date.index)]
plt.plot(baseline["Date"],baseline["Cumulative"], label = "Buy & Hold", color='tab:red')
#Strategy
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index, t in enumerate(range(int(threshold_start*100),100,5)):
    threshold = t/100
    plt.plot(Return_long_threshold.index,Return_long_threshold["Long>="+str(threshold) + " Cumulative"], label = "Long>"+str(threshold), color='tab:'+color[index])
plt.xlabel('Date')
plt.title("Cumulative  Buy & Hold  VS  Strategy 1")
plt.legend(bbox_to_anchor = (1, 1))
plt.show()

### Startegy 2
>Better Results if Long Worst News

>Hypothesis: Hype Dies Down (market rallys as a good opportunity to acquire a company expected to go back up in the future)

In [ ]:
  #Return by date and company
Ret_date_company = Invest_Data.pivot(index='Date', columns='RIC', values='RET')     

    #Lagged sent by date and company
LagSent_date_company = Invest_Data.pivot(index='Date', columns='RIC', values='Avg_Confidence(t-1)') 
LagSent_date_company.dropna(how="all", inplace=True)
#LagSent_date_company.fillna(0.5, inplace=True)  #0.5 is the proxy to neutral

    #Kepp only dates in common
relevant_dates = Ret_date_company.index.intersection(LagSent_date_company.index)
Ret_date_company = Ret_date_company.loc[relevant_dates]
LagSent_date_company = LagSent_date_company.loc[relevant_dates]

    #Average returns for companies > threshold
threshold_start = 0.2
Return_short_threshold = pd.DataFrame()
for t in range(int(threshold_start*100),0,-5):
        threshold = t/100
        Return_short_threshold["Short<="+str(threshold)] =  Ret_date_company.apply(lambda x: -np.average(x[x.index.isin(list(LagSent_date_company.loc[x.name][LagSent_date_company.loc[x.name]<=threshold].index))]) , axis=1)
        Return_short_threshold.sort_index(inplace=True)
        Return_short_threshold.fillna(0, inplace=True) #Put returns as zero (change to risk free)
        Return_short_threshold["Short<="+str(threshold) + " Cumulative"] = (Return_short_threshold["Short<="+str(threshold)]+1).cumprod()

In [ ]:
#Check Nans
Return_short_threshold[Return_short_threshold.isna().sum(axis=1)>0]

In [ ]:
#Baseline
baseline = Buy_Hold[Buy_Hold["Date"].isin(Return_by_decile_date.index)]
plt.plot(baseline["Date"],baseline["Cumulative"], label = "Buy & Hold", color='tab:red')
#Strategy
color = ["grey","blue","green","cyan","magenta","yellow","black"]
for index, t in enumerate(range(int(threshold_start*100),0,-5)):
    threshold = t/100
    plt.plot(Return_short_threshold.index,Return_short_threshold["Short<="+str(threshold) + " Cumulative"], label = "Short<="+str(threshold), color='tab:'+color[index])
plt.xlabel('Date')
plt.title("Cumulative  Buy & Hold  VS  Strategy 2")
plt.legend(bbox_to_anchor = (1, 1))
plt.show()

### Startegy 3

In [ ]:
    #Return by date and company
Ret_date_company = Invest_Data.pivot(index='Date', columns='RIC', values='RET')     

    #Lagged sent by date and company
LagSent_date_company = Invest_Data.pivot(index='Date', columns='RIC', values='Avg_Confidence(t-1)') 
LagSent_date_company.dropna(how="all", inplace=True)
#LagSent_date_company.fillna(0.5, inplace=True)  #0.5 is the proxy to neutral

    #Company decile rank by day 
n_deciles = 20
RANK_date_company_LagSent = LagSent_date_company.apply(lambda x: pd.qcut(x, n_deciles, labels=False), axis=1) 

    #Kepp only dates in common
relevant_dates = Ret_date_company.index.intersection(RANK_date_company_LagSent.index)
Ret_date_company = Ret_date_company.loc[relevant_dates]
RANK_date_company_LagSent = RANK_date_company_LagSent.loc[relevant_dates]

    #Average returns per decile
Return_by_decile_date = pd.DataFrame()
for i in range(0,n_deciles):
    #For specific date gets list of companies at given rank and then get for the same day the returns of said companies and averages them
        Return_by_decile_date["Rank("+ str(i) +")"] =  Ret_date_company.apply(lambda x: np.average(x[x.index.isin(list(RANK_date_company_LagSent.loc[x.name][RANK_date_company_LagSent.loc[x.name]==i].index))]) , axis=1)
# Stretgy returns
Return_by_decile_date["Long-Short"] = Return_by_decile_date["Rank("+ str(n_deciles-1) + ")"]  - Return_by_decile_date["Rank(0)"] 
Return_by_decile_date["Short-Long"] = Return_by_decile_date["Rank(0)"]  - Return_by_decile_date["Rank("+ str(n_deciles-1) + ")"]  
Return_by_decile_date.sort_index(inplace=True)
Return_by_decile_date["Long-Short Cumulative"] = (Return_by_decile_date["Long-Short"]+1).cumprod()
Return_by_decile_date["Short-Long Cumulative"] = (Return_by_decile_date["Short-Long"]+1).cumprod()
Return_by_decile_date.head(10)

In [ ]:
#Baseline
baseline = Buy_Hold[Buy_Hold["Date"].isin(Return_by_decile_date.index)]
plt.plot(baseline["Date"],baseline["Cumulative"], label = "Buy & Hold", color='tab:red')
#Strategy
plt.plot(Return_by_decile_date.index,Return_by_decile_date["Long-Short Cumulative"], label = "Long-Short", color='tab:blue')
plt.plot(Return_by_decile_date.index,Return_by_decile_date["Short-Long Cumulative"], label = "Short-Long", color='tab:grey')
plt.xlabel('Date')
plt.title("Cumulative  Buy & Hold  VS  Strategy 3")
plt.legend()
plt.show()

In [ ]:
Strat_Statistics = pd.DataFrame(columns=["Stats"])
Strat_Returns = Return_by_decile_date["Long-Short"].copy()
#Main Metrics
Strat_Statistics.loc["Count"] = len(Strat_Returns)
Strat_Statistics.loc[""] = ""
Strat_Statistics.loc["Mean %"] = np.average(Strat_Returns)*100*12
Strat_Statistics.loc["(mean) t-stat"] = np.average(Strat_Returns) / (np.std(Strat_Returns)/ (Strat_Statistics.loc["Count"].values[0]**0.5)) 
Strat_Statistics.loc["(mean) p-value"] = (1-norm.cdf(Strat_Statistics.loc["(mean) t-stat"].values[0]))*2
Strat_Statistics.loc["-"] = "-"
Strat_Statistics.loc["Std. Dev. (%)"] = np.round(np.std(Strat_Returns)*100*(12**0.5),2)
Strat_Statistics.loc["--"] = "--"
Strat_Statistics.loc["Sharpe ratio"] = Strat_Statistics.loc["Mean %"]/ Strat_Statistics.loc["Std. Dev. (%)"]
Strat_Statistics.loc["(SR) t-stat"] = (Strat_Statistics.loc["Sharpe ratio"].values[0]/(12**0.5))/np.sqrt((1+0.5*(Strat_Statistics.loc["Sharpe ratio"].values[0]/(12**0.5))**2)/Strat_Statistics.loc["Count"].values[0])
Strat_Statistics.loc["(SR) p-value"] = (1-norm.cdf(Strat_Statistics.loc["(SR) t-stat"].values[0]))*2
Strat_Statistics.loc["---"] = "---"
Strat_Statistics.loc["Skewness"] = skew(Strat_Returns)
Strat_Statistics.loc["Kurtosis"] = kurtosis(Strat_Returns)
Strat_Statistics.loc["----"] = "----"
Strat_Statistics.loc["JB test statistic"] = (Strat_Statistics.loc["Count"].values[0]/6)*(Strat_Statistics.loc["Skewness"].values[0]**2+0.25*(Strat_Statistics.loc["Kurtosis"].values[0]**0.5))
Strat_Statistics.loc["(JB) p-value"] = 1-chi2.cdf(Strat_Statistics.loc["JB test statistic"].values[0], 2)
Strat_Statistics.loc["-----"] = "-----"
Strat_Statistics.loc["Minimum (%)"] = Strat_Returns.min()
Strat_Statistics.loc["Percentile 25 (%)"] = np.percentile(Strat_Returns, 25)
Strat_Statistics.loc["Median (%)"] = Strat_Returns.median()
Strat_Statistics.loc["Percentile 75 (%)"] = np.percentile(Strat_Returns, 75)
Strat_Statistics.loc["Maximum (%)"] = Strat_Returns.max()
Strat_Statistics.loc["------"] = "------"
Strat_Statistics.loc["AR(1) (%)"] = Strat_Returns.corr(Strat_Returns.shift(1))
Strat_Statistics.loc["p-value"] = 2*(1-norm.cdf(Strat_Statistics.loc["Count"].values[0]**0.5*np.abs(Strat_Statistics.loc["AR(1) (%)"].values[0])))
#Displaying
display(Strat_Statistics)